In [33]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [34]:
%%R

scratch_path <- Sys.getenv("SCRATCH")
.libPaths(file.path(scratch_path, "Rlocal4.3.2"))

In [35]:
%%R
set.seed(88888888) # maximum luck

library(DESeq2)
library(plotly)
library(ggplot2)
library(viridis)
library(magrittr)
library(pheatmap)
library(DescTools)
library(pdfCluster)
library(RColorBrewer)
library(SummarizedExperiment)
library(caret)
library(class)
library(htmlwidgets)

In [36]:
%%R
OUT_DIR <- "../../../outputs/training/"
N_FOLDS <- 10

In [37]:
%%R
vst.counts <- readRDS(paste(OUT_DIR, "vst_counts.Rds", sep = ""))
print(vst.counts)

class: DESeqTransform 
dim: 9582 6288 
metadata(1): version
assays(1): ''
rownames(9582): ENSMUSG00000000001 ENSMUSG00000000028 ...
  ENSMUSG00000118462 ENSMUSG00000118541
rowData names(4): baseMean baseVar allZero dispFit
colnames(6288): V1 V2 ... V6287 V6288
colData names(3): Sample Tissue sizeFactor


In [38]:
%%R
sample <- colData(vst.counts)
print(sample)

DataFrame with 6288 rows and 3 columns
           Sample    Tissue  sizeFactor
      <character>  <factor>   <numeric>
V1             V1 Liver        0.344676
V2             V2 Brain       14.798875
V3             V3 Intestine    2.363913
V4             V4 Intestine   12.120910
V5             V5 Liver        0.754843
...           ...       ...         ...
V6284       V6284  Brain    5.56923e-04
V6285       V6285  Lung     1.25359e+01
V6286       V6286  Pancreas 3.50940e-01
V6287       V6287  Adipose  2.45864e+00
V6288       V6288  Sperm    4.92234e+00


In [39]:
%%R
vst.count.mtx <-
   vst.counts %>% SummarizedExperiment::assay() %>% as.data.frame()
print(vst.count.mtx)

                          V1        V2        V3        V4        V5        V6
ENSMUSG00000000001 15.509601 12.614846 15.519747 14.637851 14.055201 13.449191
ENSMUSG00000000028  8.444423  7.614687 10.420509  9.853538  7.905623  8.286595
ENSMUSG00000000049 20.595034  6.507014  8.077445  7.916957 20.798988  6.163516
ENSMUSG00000000058 10.526176 11.266110  8.766416 11.650305  9.692303 15.196968
ENSMUSG00000000085  9.811438 13.441422 11.548218 12.198293 11.906671 12.810130
ENSMUSG00000000088 16.480554 13.705892 14.766639 14.470708 18.266176 11.941152
ENSMUSG00000000120  9.570220  8.731973  9.042667  9.580347 11.275285  7.829239
ENSMUSG00000000125  4.150743  8.571661 10.636489  4.498000  3.916735  5.114600
ENSMUSG00000000126  7.073358  7.212174  7.601707  7.519440  7.816728 10.184149
ENSMUSG00000000127 11.007867 10.848654 10.921430 10.695660 11.936978 11.026712
ENSMUSG00000000142 10.089224 11.974981 14.516734 11.808310 10.162598 12.502983
ENSMUSG00000000149 14.462024 13.909147 12.471857 12.

ENSMUSG00000000120  8.673431  8.097015  8.534823  7.509739 14.287382  9.950898
ENSMUSG00000000125 10.691884 11.788137  9.107095  8.447594 10.539471  3.795593
ENSMUSG00000000126  7.430531 10.838424  9.501183 11.598594 10.537542  7.411331
ENSMUSG00000000127 10.858311 14.595709 11.648632 10.663082 12.409896 12.113982
ENSMUSG00000000142 11.562482 10.841656 12.306995 10.917292 14.173770 11.560112
ENSMUSG00000000149 14.777295 14.144058 14.153402 14.278215 12.319568 14.565823
ENSMUSG00000000154  5.937770  4.390378  6.871150  6.201851  5.612252 13.509429
ENSMUSG00000000168 14.229454 12.527486 13.048108 13.461763  9.063156 13.037065
ENSMUSG00000000171 13.815136 13.725101 13.049582 13.765868  9.018551 15.051932
                         V43       V44       V45       V46       V47       V48
ENSMUSG00000000001 14.863801 15.551396 15.509004 12.881099 15.083501 12.890131
ENSMUSG00000000028 10.222841  8.291743  9.959162 10.339742  8.465110  8.994179
ENSMUSG00000000049  8.770823 20.313779 19.718144  4.

ENSMUSG00000000168 12.979826 12.311295 13.708313 13.553119 13.405112 14.667263
ENSMUSG00000000171 12.931969 14.804355 16.474336 13.424510 15.704660 15.702935
                         V79       V80       V81       V82       V83       V84
ENSMUSG00000000001 12.582161 12.503299 12.640717 15.584720 12.360585 15.655105
ENSMUSG00000000028  7.948053  9.399664  9.131678  8.711479  9.039420  3.999819
ENSMUSG00000000049  8.154622  8.265488  9.322241 19.485815  7.646564 10.611460
ENSMUSG00000000058 12.562862 13.554195 13.822162 10.170051 13.003290 14.090987
ENSMUSG00000000085 11.816214 13.551970 13.467942 11.023640 12.215778 10.335457
ENSMUSG00000000088 13.873619 18.052360 17.152793 15.238424 16.525556 17.466672
ENSMUSG00000000120  6.316016  7.212498  7.577347 10.595755  8.724740  9.582843
ENSMUSG00000000125  8.244985  3.757652  5.478328  3.866418  5.409435  3.999819
ENSMUSG00000000126 11.836815 12.289930 12.020563  7.005298 11.212685 10.369105
ENSMUSG00000000127 12.502169 11.426187 11.439485 11.

ENSMUSG00000000085 13.377381 11.950843 10.682188  7.102943 11.774731  7.384592
ENSMUSG00000000088 13.748761 17.098014 14.128454 13.834484 17.289886 12.944803
ENSMUSG00000000120  8.850061  8.496343  8.831408  7.477013  9.084893  7.384592
ENSMUSG00000000125  3.836943  3.591444  8.891622 12.960932  3.682271  7.384592
ENSMUSG00000000126  8.656499 10.409242  7.762952  7.648239  8.997953  7.384592
ENSMUSG00000000127 13.637549 11.891914 11.251963 12.333782 10.935601 13.652284
ENSMUSG00000000142 10.697947 10.144484 13.998688 12.951231 11.356770 19.361243
ENSMUSG00000000149 12.747784 15.369464 11.971137 13.568616 14.846621 15.746153
ENSMUSG00000000154  8.593214  7.100619 12.085728  9.844774  8.249869  7.384592
ENSMUSG00000000168 15.018108 16.758112 12.602269 12.613951 17.571564  7.384592
ENSMUSG00000000171 13.386704 16.754294 14.942897 13.933351 17.264501 17.441888
                        V121      V122      V123      V124     V125      V126
ENSMUSG00000000001 14.743404 15.700680 12.025684 15.1

ENSMUSG00000000154 16.730378 10.771123  6.903917 11.683434  5.189046  6.299271
ENSMUSG00000000168 13.494392 14.510928 14.029953 13.278148  3.657871 13.508188
ENSMUSG00000000171 15.765562 12.158344 13.468091 13.747089  9.439626 13.214816
                        V157      V158      V159      V160      V161      V162
ENSMUSG00000000001 13.354350 13.238303 15.007388 11.390851 14.207675 14.211855
ENSMUSG00000000028  9.545080  9.728817  8.844115  9.679811  9.110544  9.520630
ENSMUSG00000000049  6.079521  5.235127  6.067967  7.860403  7.890092  6.740670
ENSMUSG00000000058 11.852679 11.818355 17.602473  8.845261 14.881035 17.624886
ENSMUSG00000000085 11.783427 11.481405 11.641590 14.091963 13.336967 11.509738
ENSMUSG00000000088 13.884197 14.158383 12.969513 12.757089 19.039605 13.464049
ENSMUSG00000000120  8.740918  8.284981  7.030062 10.070258  7.559985  7.797186
ENSMUSG00000000125  5.990732  6.291696  5.413781 10.983673  3.887093

...

ENSMUSG00000000058 10.938350 11.539889  9.826004 10.423153 12.150497 17.701679
ENSMUSG00000000085 12.807894 11.709675 10.719108 11.084535 11.901205 10.862830
ENSMUSG00000000088 12.265486 14.220229 15.464450 15.090114 13.382914 12.950228
ENSMUSG00000000120  8.158718  9.555467  8.525696 10.828864 11.514120  3.839328
ENSMUSG00000000125  8.479683  3.953795  3.663291  3.833848  6.608131  3.839328
ENSMUSG00000000126  6.682301  7.277547  7.542261  8.108540  8.011766 10.161127
ENSMUSG00000000127 12.909059 11.712215 10.570440 10.874076 12.970371 12.659989
ENSMUSG00000000142 12.182348 12.179349 10.620707 10.863363 11.612868 12.747314
ENSMUSG00000000149 13.770834 12.313218 15.147897 15.483333 11.447208 13.612709
ENSMUSG00000000154  4.082434 12.335413 14.533721 15.315064  5.870479  8.889538
ENSMUSG00000000168 12.303474 13.285836 14.391715 13.945684 13.428781 14.298359
ENSMUSG00000000171 13.145136 14.703320 16.873866 16.427229 12.766311 14.005713
                       V6151     V6152     V6153    

ENSMUSG00000000142  9.143509 11.948458 10.942205 11.848770 12.470772 11.696861
ENSMUSG00000000149 13.358113 12.764481 14.081593 13.659464 15.000711 13.320412
ENSMUSG00000000154 16.540062  3.722818  6.502334  3.864646  5.340178  9.263673
ENSMUSG00000000168 15.937029 13.379436 13.179568 12.550707 14.513662 12.781874
ENSMUSG00000000171 19.127373 11.790645 13.224631 12.712022 14.355157 13.097670
                       V6187     V6188     V6189     V6190     V6191     V6192
ENSMUSG00000000001 14.879085 12.144884 13.564916 14.385837 12.744001 15.282920
ENSMUSG00000000028  8.560346  8.409413  9.036266 10.259634  9.448997  8.446501
ENSMUSG00000000049 20.211238  8.769209  8.300899  5.626939 15.942785 20.510194
ENSMUSG00000000058  9.982504 11.479532 15.008995 16.931556  8.863129 10.360405
ENSMUSG00000000085 11.073364 14.659624 12.548245 11.260643 12.425748 11.461141
ENSMUSG00000000088 14.570018 12.538853 12.230861 18.266405 10.292995 16.570292
ENSMUSG00000000120  9.049308  8.044307 13.542916  5.

ENSMUSG00000000049 15.883219  9.068958 19.662210  6.115806 20.139153 20.599467
ENSMUSG00000000058  5.844850  3.930394 10.768600 14.358336  8.564371 10.561304
ENSMUSG00000000085 13.431321 10.455396 11.280443 12.141798 11.183213 10.816274
ENSMUSG00000000088 19.558055 18.490656 15.261954 12.226430 15.547942 14.714769
ENSMUSG00000000120  9.294799 12.971854  8.987832  9.641350 10.812571 10.371292
ENSMUSG00000000125 10.153494 10.708937  3.738736  3.517273  3.572489  3.924014
ENSMUSG00000000126  5.346454 14.702688  9.285470  8.072537  7.380869  7.627529
ENSMUSG00000000127 15.589118 10.503867 11.089956 10.345656 10.702604 11.832752
ENSMUSG00000000142  6.778793 17.680289 10.962781 11.191588 11.908074 11.466501
ENSMUSG00000000149 10.967139  3.930394 15.496008 14.203556 15.343062 16.280472
ENSMUSG00000000154 12.091632  3.930394 14.355025 10.042424 15.417119 14.020496
ENSMUSG00000000168 15.359548 15.365176 13.718796 12.610471 14.790290 14.249813
ENSMUSG00000000171 16.174521 13.976896 16.139202 13.

ENSMUSG00000000127 11.825896 11.894054 11.216631  9.459318 11.313645 11.048332
ENSMUSG00000000142 11.678095 10.877561  9.333552 10.482774 10.159872 11.592164
ENSMUSG00000000149 13.751716 12.778124 15.096372 13.593410 14.208269 15.412342
ENSMUSG00000000154 10.061063  6.336885  6.537522  7.378698  9.422719 14.033745
ENSMUSG00000000168 15.987226 15.485720 14.426306 11.738116 14.667429 14.438883
ENSMUSG00000000171 15.633356 15.103295 15.367482 13.155931 15.122590 16.088813
                       V6265     V6266     V6267     V6268     V6269     V6270
ENSMUSG00000000001 14.740222 13.953879 15.420441 14.448874 15.944623 15.627848
ENSMUSG00000000028  9.506858 10.567309 10.466259 10.632327  9.963147 10.698355
ENSMUSG00000000049 20.243781  6.729774  7.225307  5.456175  8.095959  6.024538
ENSMUSG00000000058 11.018555 12.536804 12.560725 11.646309 10.505353 11.506812
ENSMUSG00000000085 11.586213 11.250557

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [40]:
%%R
GEO_model_training_tissue_detail.vec <-
   readRDS(paste(OUT_DIR, "GEO_model_training_tissue_detail_vec.Rds", sep = ""))
print(GEO_model_training_tissue_detail.vec)

   [1] "Liver"         "Brain"         "Intestine"     "Intestine"    
   [5] "Liver"         "Lung"          "Intestine"     "Liver"        
   [9] "Brain"         "Muscle"        "Liver"         "Liver"        
  [13] "Liver"         "Liver"         "Liver"         "Liver"        
  [17] "Brain"         "Muscle"        "Liver"         "Liver"        
  [21] "Testes"        "Heart"         "Skin"          "Brain"        
  [25] "Liver"         "Liver"         "Kidney"        "Testes"       
  [29] "Muscle"        "Skin"          "Intestine"     "Uterus"       
  [33] "Uterus"        "Liver"         "Muscle"        "Liver"        
  [37] "Brain"         "Sperm"         "Testes"        "Brain"        
  [41] "Eye"           "Liver"         "Intestine"     "Liver"        
  [45] "Liver"         "Adipose"       "Liver"         "Muscle"       
  [49] "Heart"         "Adipose"       "Lung"          "Eye"          
  [53] "Liver"         "Liver"         "Adipose"       "Adrenal"      
  [57]

 [461] "Brain"         "Ovary"         "Liver"         "Kidney"       
 [465] "Pancreas"      "Adipose"       "Intestine"     "Kidney"       
 [469] "Liver"         "Brain"         "Lung"          "Liver"        
 [473] "Brain"         "Intestine"     "Intestine"     "Liver"        
 [477] "Kidney"        "Adipose"       "Liver"         "Liver"        
 [481] "Liver"         "Muscle"        "Liver"         "Liver"        
 [485] "Liver"         "Muscle"        "Adipose"       "Adipose"      
 [489] "Adipose"       "MOE"           "Liver"         "Liver"        
 [493] "Adipose"       "Olfactory"     "Testes"        "Lung"         
 [497] "Testes"        "Liver"         "Kidney"        "Brain"        
 [501] "Liver"         "Uterus"        "Brain"         "Brain"        
 [505] "Liver"         "Kidney"        "Intestine"     "Liver"        
 [509] "Liver"         "Adipose"       "Liver"         "Intestine"    
 [513] "Liver"         "Brain"         "Liver"         "Adipose"      
 [517]

 [921] "Brain"         "Muscle"        "Kidney"        "Brain"        
 [925] "Brain"         "Testes"        "Brain"         "Heart"        
 [929] "Liver"         "Adipose"       "Liver"         "Intestine"    
 [933] "Brain"         "Adipose"       "Liver"         "Brain"        
 [937] "Brain"         "Muscle"        "Liver"         "Muscle"       
 [941] "Liver"         "Bone"          "Heart"         "Brain"        
 [945] "Testes"        "Muscle"        "Eye"           "Liver"        
 [949] "Sperm"         "Liver"         "Uterus"        "Kidney"       
 [953] "Adipose"       "Intestine"     "Skin"          "Brain"        
 [957] "Kidney"        "Liver"         "Brain"         "Intestine"    
 [961] "Liver"         "Brain"         "VNO"           "Olfactory"    
 [965] "Liver"         "Liver"         "Brain"         "Sperm"        
 [969] "Brain"         "Liver"         "Liver"         "Liver"        
 [973] "Liver"         "Intestine"     "Intestine"     "Brain"        
 [977]

[1381] "Brain"         "Liver"         "Liver"         "Liver"        
[1385] "Skin"          "Testes"        "Liver"         "Brain"        
[1389] "Brain"         "Adipose"       "Brain"         "Muscle"       
[1393] "Adipose"       "Testes"        "Heart"         "Intestine"    
[1397] "Intestine"     "Brain"         "Skin"          "Muscle"       
[1401] "Muscle"        "Brain"         "Liver"         "Lung"         
[1405] "Liver"         "Brain"         "Muscle"        "Bone"         
[1409] "Liver"         "Brain"         "Sperm"         "Lung"         
[1413] "Liver"         "MOE"           "Liver"         "Brain"        
[1417] "Brain"         "Brain"         "Liver"         "Liver"        
[1421] "Adipose"       "Adipose"       "Liver"         "Bone"         
[1425] "Bone"          "Liver"         "Brain"         "Brain"        
[1429] "Liver"         "Pancreas"      "Liver"         "Pancreas"     
[1433] "Pancreas"      "Liver"         "Pancreas"      "Adipose"      
[1437]

[1841] "Brain"         "Adipose"       "Sperm"         "Heart"        
[1845] "Ovary"         "Nerve"         "Heart"         "Brain"        
[1849] "Adipose"       "Brain"         "Intestine"     "Testes"       
[1853] "Eye"           "Liver"         "Nerve"         "Brain"        
[1857] "Liver"         "Muscle"        "Spinal cord"   "Lung"         
[1861] "Liver"         "Liver"         "Kidney"        "Liver"        
[1865] "Intestine"     "Adipose"       "Kidney"        "Brain"        
[1869] "Adipose"       "Intestine"     "Pancreas"      "Testes"       
[1873] "Liver"         "Ovary"         "Liver"         "Adipose"      
[1877] "Liver"         "Liver"         "MOE"           "Brain"        
[1881] "Brain"         "Kidney"        "Brain"         "Nerve"        
[1885] "Brain"         "Liver"         "Brain"         "Kidney"       
[1889] "Testes"        "Intestine"     "Liver"         "Stomach"      
[1893] "Intestine"     "Ovary"         "Adipose"       "Pancreas"     
[1897]

...

[4677] "Lung"          "Testes"        "Muscle"        "Brain"        
[4681] "Brain"         "Skin"          "Adipose"       "MOE"          
[4685] "Pancreas"      "Liver"         "Brain"         "Liver"        
[4689] "Pancreas"      "Spinal cord"   "Pancreas"      "Testes"       
[4693] "Adipose"       "Brain"         "Kidney"        "Intestine"    
[4697] "VNO"           "Adipose"       "Muscle"        "Brain"        
[4701] "Liver"         "Liver"         "Pancreas"      "Liver"        
[4705] "Spinal cord"   "Brain"         "Muscle"        "Pancreas"     
[4709] "Intestine"     "Intestine"     "Liver"         "Adipose"      
[4713] "Pancreas"      "Kidney"        "Adipose"       "Eye"          
[4717] "Brain"         "Liver"         "Liver"         "Brain"        
[4721] "Liver"         "Muscle"        "Intestine"     "Kidney"       
[4725] "Liver"         "Intestine"     "Intestine"     "Liver"        
[4729] "Olfactory"     "Liver"         "Brain"         "Muscle"       
[4733]

[5137] "Muscle"        "Intestine"     "Brain"         "Lung"         
[5141] "Muscle"        "Heart"         "Liver"         "Brain"        
[5145] "Adipose"       "Testes"        "Brain"         "Kidney"       
[5149] "Kidney"        "Intestine"     "Kidney"        "Liver"        
[5153] "Muscle"        "Eye"           "Brain"         "Brain"        
[5157] "Pancreas"      "Liver"         "Brain"         "Brain"        
[5161] "Brain"         "Muscle"        "Liver"         "Brain"        
[5165] "Liver"         "Brain"         "Lung"          "Intestine"    
[5169] "Intestine"     "Lung"          "Nerve"         "Lung"         
[5173] "Liver"         "Adipose"       "Kidney"        "Brain"        
[5177] "Heart"         "Liver"         "Adipose"       "Tendon"       
[5181] "Adipose"       "Heart"         "Liver"         "Uterus"       
[5185] "Liver"         "Heart"         "Liver"         "Adipose"      
[5189] "Kidney"        "Liver"         "Lung"          "Liver"        
[5193]

[5597] "Brain"         "Heart"         "Liver"         "Liver"        
[5601] "Kidney"        "Intestine"     "Skin"          "Pancreas"     
[5605] "Intestine"     "Aorta"         "Muscle"        "Liver"        
[5609] "Tendon"        "Intestine"     "Brain"         "Skin"         
[5613] "Brain"         "Kidney"        "Heart"         "Kidney"       
[5617] "Lung"          "Liver"         "Eye"           "Kidney"       
[5621] "Adipose"       "Skin"          "Lung"          "Adipose"      
[5625] "Muscle"        "Brain"         "Testes"        "Spinal cord"  
[5629] "Muscle"        "Lung"          "Heart"         "Intestine"    
[5633] "Kidney"        "Liver"         "Kidney"        "Testes"       
[5637] "Testes"        "Adipose"       "Liver"         "Intestine"    
[5641] "Brain"         "Liver"         "Adipose"       "Tendon"       
[5645] "Brain"         "Sperm"         "Heart"         "Liver"        
[5649] "Kidney"        "Liver"         "Eye"           "Brain"        
[5653]

[6057] "Brain"         "Heart"         "VNO"           "Heart"        
[6061] "VNO"           "Brain"         "Brain"         "Intestine"    
[6065] "Muscle"        "Lung"          "Heart"         "Ovary"        
[6069] "Liver"         "Liver"         "Heart"         "Liver"        
[6073] "Muscle"        "Liver"         "Pancreas"      "Lung"         
[6077] "Muscle"        "Aorta"         "Liver"         "MOE"          
[6081] "Liver"         "Eye"           "Brain"         "Lung"         
[6085] "Liver"         "Kidney"        "Brain"         "Testes"       
[6089] "Liver"         "Brain"         "Brain"         "Liver"        
[6093] "Lung"          "Kidney"        "Brain"         "Liver"        
[6097] "Testes"        "Sperm"         "Brain"         "Liver"        
[6101] "VNO"           "Brain"         "Liver"         "Muscle"       
[6105] "Brain"         "Brain"         "Adipose"       "Stomach"      
[6109] "Liver"         "Adipose"       "Liver"         "Adipose"      
[6113]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [41]:
%%R
rxn2ensembls.nls <-
   readRDS(paste(OUT_DIR, "rxn2ensembls_nls.Rds", sep = ""))
print(rxn2ensembls.nls)

$`R-MMU-170666`
 [1] "ENSMUSG00000000001" "ENSMUSG00000005580" "ENSMUSG00000020431"
 [4] "ENSMUSG00000020654" "ENSMUSG00000021536" "ENSMUSG00000022220"
 [7] "ENSMUSG00000022376" "ENSMUSG00000022840" "ENSMUSG00000022994"
[10] "ENSMUSG00000024524" "ENSMUSG00000028777" "ENSMUSG00000031659"
[13] "ENSMUSG00000032562" "ENSMUSG00000057614"

$`R-MMU-170671`
 [1] "ENSMUSG00000000001" "ENSMUSG00000005580" "ENSMUSG00000020431"
 [4] "ENSMUSG00000020654" "ENSMUSG00000021536" "ENSMUSG00000022220"
 [7] "ENSMUSG00000022376" "ENSMUSG00000022840" "ENSMUSG00000022994"
[10] "ENSMUSG00000024524" "ENSMUSG00000028777" "ENSMUSG00000031659"
[13] "ENSMUSG00000032562" "ENSMUSG00000057614"

$`R-MMU-170674`
 [1] "ENSMUSG00000000001" "ENSMUSG00000005580" "ENSMUSG00000020431"
 [4] "ENSMUSG00000020654" "ENSMUSG00000021536" "ENSMUSG00000022220"
 [7] "ENSMUSG00000022376" "ENSMUSG00000022840" "ENSMUSG00000022994"
[10] "ENSMUSG00000024524" "ENSMUSG00000028777" "ENSMUSG00000031659"
[13] "ENSMUSG00000032562" "ENSMUSG000000

 [22] "ENSMUSG00000023078" "ENSMUSG00000023192" "ENSMUSG00000023235"
 [25] "ENSMUSG00000023439" "ENSMUSG00000024164" "ENSMUSG00000024211"
 [28] "ENSMUSG00000024462" "ENSMUSG00000024553" "ENSMUSG00000024659"
 [31] "ENSMUSG00000024907" "ENSMUSG00000025496" "ENSMUSG00000025739"
 [34] "ENSMUSG00000025804" "ENSMUSG00000025905" "ENSMUSG00000026166"
 [37] "ENSMUSG00000026180" "ENSMUSG00000026237" "ENSMUSG00000026424"
 [40] "ENSMUSG00000026874" "ENSMUSG00000027400" "ENSMUSG00000027584"
 [43] "ENSMUSG00000027669" "ENSMUSG00000027762" "ENSMUSG00000028004"
 [46] "ENSMUSG00000028738" "ENSMUSG00000028777" "ENSMUSG00000028950"
 [49] "ENSMUSG00000028971" "ENSMUSG00000029064" "ENSMUSG00000029072"
 [52] "ENSMUSG00000029236" "ENSMUSG00000029371" "ENSMUSG00000029372"
 [55] "ENSMUSG00000029373" "ENSMUSG00000029379" "ENSMUSG00000029380"
 [58] "ENSMUSG00000029417" "ENSMUSG00000029530" "ENSMUSG00000029663"
 [61] "ENSMUSG00000029713" "ENSMUSG00000029819" "ENSMUSG00000031778"
 [64] "ENSMUSG00000031861" "ENSMUS

[175] "ENSMUSG00000063594" "ENSMUSG00000066090" "ENSMUSG00000067586"
[178] "ENSMUSG00000067604" "ENSMUSG00000067714" "ENSMUSG00000068122"
[181] "ENSMUSG00000068523" "ENSMUSG00000070687" "ENSMUSG00000071005"
[184] "ENSMUSG00000071147" "ENSMUSG00000071150" "ENSMUSG00000071230"
[187] "ENSMUSG00000071275" "ENSMUSG00000071276" "ENSMUSG00000071311"
[190] "ENSMUSG00000071658" "ENSMUSG00000073877" "ENSMUSG00000073878"
[193] "ENSMUSG00000073888" "ENSMUSG00000074715" "ENSMUSG00000079227"
[196] "ENSMUSG00000079700" "ENSMUSG00000094065" "ENSMUSG00000094686"
[199] "ENSMUSG00000095675" "ENSMUSG00000096596" "ENSMUSG00000096826"
[202] "ENSMUSG00000114755"

$`R-MMU-751001`
 [1] "ENSMUSG00000000001" "ENSMUSG00000021303" "ENSMUSG00000023439"
 [4] "ENSMUSG00000025739" "ENSMUSG00000027669" "ENSMUSG00000028777"
 [7] "ENSMUSG00000029064" "ENSMUSG00000029663" "ENSMUSG00000029713"
[10] "ENSMUSG00000032192" "ENSMUSG00000032562" "ENSMUSG00000032766"
[13] "ENSMUSG00000036402" "ENSMUSG00000038607" "ENSMUSG00000038

 [7] "ENSMUSG00000024910" "ENSMUSG00000025784" "ENSMUSG00000026295"
[10] "ENSMUSG00000028108" "ENSMUSG00000029672" "ENSMUSG00000033880"
[13] "ENSMUSG00000064373"

$`R-MMU-9021068`
[1] "ENSMUSG00000000058" "ENSMUSG00000007655" "ENSMUSG00000019768"
[4] "ENSMUSG00000021055"

$`R-MMU-9021079`
[1] "ENSMUSG00000000058" "ENSMUSG00000007655" "ENSMUSG00000019768"
[4] "ENSMUSG00000021055"

$`R-MMU-3108203`
 [1] "ENSMUSG00000000085" "ENSMUSG00000015120" "ENSMUSG00000018537"
 [4] "ENSMUSG00000020265" "ENSMUSG00000020738" "ENSMUSG00000024325"
 [7] "ENSMUSG00000025577" "ENSMUSG00000025578" "ENSMUSG00000026484"
[10] "ENSMUSG00000026739" "ENSMUSG00000028796" "ENSMUSG00000037373"
[13] "ENSMUSG00000037652" "ENSMUSG00000039989" "ENSMUSG00000040669"

$`R-MMU-3108209`
 [1] "ENSMUSG00000000085" "ENSMUSG00000015120" "ENSMUSG00000018537"
 [4] "ENSMUSG00000024325" "ENSMUSG00000025577" "ENSMUSG00000025578"
 [7] "ENSMUSG00000026021" "ENSMUSG00000026484" "ENSMUSG00000026739"
[10] "ENSMUSG00000028796" "ENSMUSG0000

 [1] "ENSMUSG00000000125" "ENSMUSG00000000126" "ENSMUSG00000009900"
 [4] "ENSMUSG00000010797" "ENSMUSG00000012282" "ENSMUSG00000015957"
 [7] "ENSMUSG00000018486" "ENSMUSG00000021994" "ENSMUSG00000022382"
[10] "ENSMUSG00000022996" "ENSMUSG00000022997" "ENSMUSG00000026167"
[13] "ENSMUSG00000027840" "ENSMUSG00000029671" "ENSMUSG00000030093"
[16] "ENSMUSG00000030170" "ENSMUSG00000033227" "ENSMUSG00000036856"
[19] "ENSMUSG00000036961" "ENSMUSG00000063406"

$`R-MMU-3247840`
 [1] "ENSMUSG00000000125" "ENSMUSG00000000126" "ENSMUSG00000009900"
 [4] "ENSMUSG00000010797" "ENSMUSG00000012282" "ENSMUSG00000015957"
 [7] "ENSMUSG00000018486" "ENSMUSG00000021994" "ENSMUSG00000022382"
[10] "ENSMUSG00000022996" "ENSMUSG00000022997" "ENSMUSG00000026167"
[13] "ENSMUSG00000027840" "ENSMUSG00000028173" "ENSMUSG00000029671"
[16] "ENSMUSG00000030093" "ENSMUSG00000030170" "ENSMUSG00000033227"
[19] "ENSMUSG00000036856" "ENSMUSG00000036961"

$`R-MMU-3247843`
 [1] "ENSMUSG00000000125" "ENSMUSG00000000126" "ENSMUS

...

[1] "ENSMUSG00000032725"

$`R-MMU-8942344`
[1] "ENSMUSG00000032725"

$`R-MMU-2685505`
[1] "ENSMUSG00000032741" "ENSMUSG00000048677"

$`R-MMU-425822`
[1] "ENSMUSG00000032754"

$`R-MMU-8949688`
[1] "ENSMUSG00000032754"

$`R-MMU-9701211`
[1] "ENSMUSG00000032769"

$`R-MMU-390649`
[1] "ENSMUSG00000032773" "ENSMUSG00000046159" "ENSMUSG00000074939"

$`R-MMU-9704151`
[1] "ENSMUSG00000032773" "ENSMUSG00000046159" "ENSMUSG00000074939"

$`R-MMU-964958`
[1] "ENSMUSG00000032788"

$`R-MMU-964962`
[1] "ENSMUSG00000032788"

$`R-MMU-964970`
[1] "ENSMUSG00000032788"

$`R-MMU-2161950`
[1] "ENSMUSG00000032807"

$`R-MMU-8848864`
[1] "ENSMUSG00000032812" "ENSMUSG00000038751"

$`R-MMU-8848873`
[1] "ENSMUSG00000032812" "ENSMUSG00000038751"

$`R-MMU-429157`
[1] "ENSMUSG00000032849"

$`R-MMU-9750546`
[1] "ENSMUSG00000032849"

$`R-MMU-6785933`
[1] "ENSMUSG00000032854"

$`R-MMU-417927`
[1] "ENSMUSG00000032860"

$`R-MMU-5682010`
[1] "ENSMUSG00000032905" "ENSMUSG00000038160"

$`R-MMU-5682690`
[1] "ENSMUSG0000003290

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



$`R-MMU-8858270`
[1] "ENSMUSG00000043421"

$`R-MMU-5605262`
[1] "ENSMUSG00000043648"

$`R-MMU-444620`
[1] "ENSMUSG00000043659" "ENSMUSG00000073804"

$`R-MMU-9009950`
[1] "ENSMUSG00000043702"

$`R-MMU-8937744`
[1] "ENSMUSG00000043733"

$`R-MMU-8937767`
[1] "ENSMUSG00000043733"

$`R-MMU-914067`
[1] "ENSMUSG00000043733" "ENSMUSG00000059923"

$`R-MMU-9604980`
[1] "ENSMUSG00000043733" "ENSMUSG00000110206"

$`R-MMU-9606789`
[1] "ENSMUSG00000043733" "ENSMUSG00000059923" "ENSMUSG00000110206"

$`R-MMU-8870352`
[1] "ENSMUSG00000043885"

$`R-MMU-8870354`
[1] "ENSMUSG00000043885"

$`R-MMU-419428`
[1] "ENSMUSG00000043895" "ENSMUSG00000044199" "ENSMUSG00000045087"
[4] "ENSMUSG00000067586"

$`R-MMU-3296301`
[1] "ENSMUSG00000043953" "ENSMUSG00000071005"

$`R-MMU-9830451`
[1] "ENSMUSG00000043969"

$`R-MMU-975829`
[1] "ENSMUSG00000043998"

$`R-MMU-373086`
[1] "ENSMUSG00000044052" "ENSMUSG00000073877" "ENSMUSG00000073888"
[4] "ENSMUSG00000074715" "ENSMUSG00000096826"

$`R-MMU-211178`
[1] "ENSMUSG00000044

[1] "ENSMUSG00000046971" "ENSMUSG00000070719"

$`R-MMU-9794031`
[1] "ENSMUSG00000047002"

$`R-MMU-437240`
[1] "ENSMUSG00000047139"

$`R-MMU-140355`
[1] "ENSMUSG00000047250"

$`R-MMU-140359`
[1] "ENSMUSG00000047250"

$`R-MMU-2314678`
[1] "ENSMUSG00000047250"

$`R-MMU-9677320`
[1] "ENSMUSG00000047250"

$`R-MMU-6803890`
[1] "ENSMUSG00000047281"

$`R-MMU-947531`
[1] "ENSMUSG00000047454"

$`R-MMU-5340195`
[1] "ENSMUSG00000047638"

$`R-MMU-6806674`
[1] "ENSMUSG00000047719"

$`R-MMU-190681`
[1] "ENSMUSG00000047797"

$`R-MMU-9846477`
[1] "ENSMUSG00000047878"

$`R-MMU-9825608`
[1] "ENSMUSG00000047897"

$`R-MMU-9825624`
[1] "ENSMUSG00000047897"

$`R-MMU-8850846`
[1] "ENSMUSG00000048120"

$`R-MMU-8850854`
[1] "ENSMUSG00000048120"

$`R-MMU-8954468`
[1] "ENSMUSG00000048142"

$`R-MMU-70542`
[1] "ENSMUSG00000048217"

$`R-MMU-6804402`
[1] "ENSMUSG00000048349" "ENSMUSG00000059552"

$`R-MMU-418172`
[1] "ENSMUSG00000048376" "ENSMUSG00000060216"

$`R-MMU-9708866`
[1] "ENSMUSG00000048376"

$`R-MMU-9006144`

[1] "ENSMUSG00000053460" "ENSMUSG00000074486"

$`R-MMU-177784`
[1] "ENSMUSG00000053604"

$`R-MMU-71306`
[1] "ENSMUSG00000053604"

$`R-MMU-6797955`
[1] "ENSMUSG00000053644"

$`R-MMU-70941`
[1] "ENSMUSG00000053644"

$`R-MMU-374207`
[1] "ENSMUSG00000053647"

$`R-MMU-8980228`
[1] "ENSMUSG00000053846"

$`R-MMU-6787533`
[1] "ENSMUSG00000053870"

$`R-MMU-4084989`
[1] "ENSMUSG00000053916"

$`R-MMU-9708104`
[1] "ENSMUSG00000053965"

$`R-MMU-5688294`
[1] "ENSMUSG00000054021" "ENSMUSG00000058927" "ENSMUSG00000063694"

$`R-MMU-8850797`
[1] "ENSMUSG00000054046" "ENSMUSG00000055095"

$`R-MMU-444204`
[1] "ENSMUSG00000054200"

$`R-MMU-9842656`
[1] "ENSMUSG00000054204" "ENSMUSG00000055471" "ENSMUSG00000087247"

$`R-MMU-9011343`
[1] "ENSMUSG00000054256"

$`R-MMU-9816416`
[1] "ENSMUSG00000054272" "ENSMUSG00000070828" "ENSMUSG00000090714"

$`R-MMU-9820700`
[1] "ENSMUSG00000054272" "ENSMUSG00000054310" "ENSMUSG00000055942"
[4] "ENSMUSG00000070828" "ENSMUSG00000090714" "ENSMUSG00000099216"
[7] "ENSMUSG00000

In [42]:
%%R
rxns <- rxn2ensembls.nls %>% names()
print(rxns)

   [1] "R-MMU-170666"  "R-MMU-170671"  "R-MMU-170674"  "R-MMU-170686" 
   [5] "R-MMU-380073"  "R-MMU-392187"  "R-MMU-392195"  "R-MMU-392202" 
   [9] "R-MMU-749454"  "R-MMU-749456"  "R-MMU-751001"  "R-MMU-8964229"
  [13] "R-MMU-8964242" "R-MMU-8964329" "R-MMU-8982012" "R-MMU-8982019"
  [17] "R-MMU-8982020" "R-MMU-9021596" "R-MMU-9021600" "R-MMU-9021601"
  [21] "R-MMU-176298"  "R-MMU-176318"  "R-MMU-68914"   "R-MMU-68916"  
  [25] "R-MMU-68917"   "R-MMU-68960"   "R-MMU-481009"  "R-MMU-9021068"
  [29] "R-MMU-9021079" "R-MMU-3108203" "R-MMU-3108209" "R-MMU-3927824"
  [33] "R-MMU-4551655" "R-MMU-4551727" "R-MMU-4570463" "R-MMU-4570499"
  [37] "R-MMU-8937989" "R-MMU-163214"  "R-MMU-9709406" "R-MMU-9838627"
  [41] "R-MMU-9839113" "R-MMU-193641"  "R-MMU-193646"  "R-MMU-193650" 
  [45] "R-MMU-193653"  "R-MMU-193656"  "R-MMU-193665"  "R-MMU-193668" 
  [49] "R-MMU-193669"  "R-MMU-193679"  "R-MMU-193682"  "R-MMU-193684" 
  [53] "R-MMU-193694"  "R-MMU-193695"  "R-MMU-193696"  "R-MMU-193700" 
  [57]

[1133] "R-MMU-8852298" "R-MMU-8852306" "R-MMU-8865774" "R-MMU-8867370"
[1137] "R-MMU-9610428" "R-MMU-9610641" "R-MMU-9646393" "R-MMU-9646679"
[1141] "R-MMU-9646685" "R-MMU-9648017" "R-MMU-9648114" "R-MMU-9668419"
[1145] "R-MMU-983259"  "R-MMU-983266"  "R-MMU-9836404" "R-MMU-3769397"
[1149] "R-MMU-9832124" "R-MMU-9832199" "R-MMU-114577"  "R-MMU-114600" 
[1153] "R-MMU-114670"  "R-MMU-114671"  "R-MMU-139842"  "R-MMU-1454757"
[1157] "R-MMU-1454822" "R-MMU-1474266" "R-MMU-2002428" "R-MMU-2002440"
[1161] "R-MMU-216045"  "R-MMU-2213201" "R-MMU-2213205" "R-MMU-2243926"
[1165] "R-MMU-2243931" "R-MMU-2250301" "R-MMU-2327733" "R-MMU-2327738"
[1169] "R-MMU-2395223" "R-MMU-2395257" "R-MMU-2395302" "R-MMU-2395314"
[1173] "R-MMU-2395322" "R-MMU-2395324" "R-MMU-2424243" "R-MMU-2424252"
[1177] "R-MMU-2465890" "R-MMU-2473596" "R-MMU-429529"  "R-MMU-430073" 
[1181] "R-MMU-430076"  "R-MMU-430096"  "R-MMU-437118"  "R-MMU-443402" 
[1185] "R-MMU-443418"  "R-MMU-453183"  "R-MMU-453200"  "R-MMU-5696356"
[1189]

[1593] "R-MMU-8850356" "R-MMU-8866277" "R-MMU-112392"  "R-MMU-112395" 
[1597] "R-MMU-112396"  "R-MMU-113411"  "R-MMU-113412"  "R-MMU-113413" 
[1601] "R-MMU-113414"  "R-MMU-113429"  "R-MMU-1363303" "R-MMU-1363306"
[1605] "R-MMU-1363311" "R-MMU-1363314" "R-MMU-157906"  "R-MMU-187520" 
[1609] "R-MMU-187552"  "R-MMU-187574"  "R-MMU-187575"  "R-MMU-188350" 
[1613] "R-MMU-188386"  "R-MMU-188390"  "R-MMU-69005"   "R-MMU-69191"  
[1617] "R-MMU-69195"   "R-MMU-69199"   "R-MMU-69562"   "R-MMU-8848414"
[1621] "R-MMU-8848436" "R-MMU-9706390" "R-MMU-390593"  "R-MMU-390595" 
[1625] "R-MMU-390597"  "R-MMU-390598"  "R-MMU-1168640" "R-MMU-1234159"
[1629] "R-MMU-1236970" "R-MMU-1504193" "R-MMU-209061"  "R-MMU-2130282"
[1633] "R-MMU-264458"  "R-MMU-353125"  "R-MMU-4608855" "R-MMU-4641260"
[1637] "R-MMU-5362448" "R-MMU-5607724" "R-MMU-5607731" "R-MMU-5610754"
[1641] "R-MMU-5610758" "R-MMU-5610760" "R-MMU-5635854" "R-MMU-5635868"
[1645] "R-MMU-5658430" "R-MMU-5665854" "R-MMU-5665871" "R-MMU-5668481"
[1649]

[2053] "R-MMU-187506"  "R-MMU-187916"  "R-MMU-187934"  "R-MMU-5671980"
[2057] "R-MMU-9632868" "R-MMU-9632873" "R-MMU-9699578" "R-NUL-9618292"
[2061] "R-MMU-442516"  "R-MMU-9625402" "R-NUL-9625422" "R-MMU-2130619"
[2065] "R-MMU-2130641" "R-MMU-2213236" "R-MMU-432688"  "R-MMU-432706" 
[2069] "R-MMU-432707"  "R-MMU-432712"  "R-MMU-5333658" "R-MMU-8951498"
[2073] "R-MMU-1449633" "R-MMU-2255341" "R-MMU-2562526" "R-MMU-5623519"
[2077] "R-MMU-5623521" "R-MMU-5623524" "R-MMU-5623525" "R-MMU-5623527"
[2081] "R-MMU-8854182" "R-MMU-9021357" "R-MMU-211910"  "R-MMU-211929" 
[2085] "R-MMU-211988"  "R-MMU-212004"  "R-MMU-196126"  "R-MMU-200421" 
[2089] "R-MMU-3222006" "R-MMU-3222020" "R-MMU-380942"  "R-MMU-6805035"
[2093] "R-MMU-3928646" "R-MMU-3928651" "R-MMU-3928661" "R-MMU-4093327"
[2097] "R-MMU-5683405" "R-MMU-8939204" "R-MMU-9709547" "R-MMU-9716913"
[2101] "R-MMU-9716947" "R-MMU-6797090" "R-MMU-8848484" "R-MMU-429955" 
[2105] "R-MMU-6798044" "R-MMU-9817704" "R-MMU-9819258" "R-MMU-9822325"
[2109]

[2513] "R-MMU-9014449" "R-MMU-109700"  "R-MMU-109702"  "R-MMU-1449591"
[2517] "R-MMU-1458507" "R-MMU-5263624" "R-MMU-189398"  "R-MMU-1564164"
[2521] "R-MMU-2396370" "R-MMU-8944223" "R-MMU-212222"  "R-MMU-212269" 
[2525] "R-MMU-4655431" "R-MMU-573336"  "R-MMU-573376"  "R-MMU-573383" 
[2529] "R-MMU-1605591" "R-MMU-1605632" "R-MMU-1605724" "R-MMU-1605736"
[2533] "R-MMU-1606312" "R-MMU-1606564" "R-MMU-1606602" "R-MMU-1606807"
[2537] "R-MMU-351341"  "R-MMU-5336182" "R-MMU-5336184" "R-MMU-9840334"
[2541] "R-MMU-9840470" "R-MMU-9840488" "R-MMU-9841189" "R-MMU-9842115"
[2545] "R-MMU-1168638" "R-MMU-168140"  "R-MMU-202500"  "R-MMU-202534" 
[2549] "R-MMU-202541"  "R-MMU-2730876" "R-MMU-450346"  "R-MMU-451561" 
[2553] "R-MMU-4755411" "R-MMU-5357776" "R-MMU-5357831" "R-MMU-5607742"
[2557] "R-MMU-5609665" "R-MMU-5626962" "R-MMU-5684267" "R-MMU-5684275"
[2561] "R-MMU-5696627" "R-MMU-622415"  "R-MMU-688136"  "R-MMU-688137" 
[2565] "R-MMU-688985"  "R-MMU-706479"  "R-MMU-741386"  "R-MMU-741411" 
[2569]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 "R-MMU-4086410"
[4821] "R-MMU-5195402" "R-MMU-5244669" "R-MMU-5692755" "R-MMU-6793661"
[4825] "R-MMU-6803875" "R-MMU-8940100" "R-MMU-9009282" "R-MMU-9836184"
[4829] "R-MMU-418309"  "R-MMU-9018774" "R-MMU-4724284" "R-MMU-5661115"
[4833] "R-MMU-5675376" "R-MMU-1433395" "R-MMU-6795290" "R-MMU-6795460"
[4837] "R-MMU-6795473" "R-MMU-9022765" "R-MMU-9022767" "R-MMU-9612501"
[4841] "R-MMU-430028"  "R-MMU-6797553" "R-MMU-6797568" "R-MMU-114252" 
[4845] "R-MMU-114254"  "R-MMU-114259"  "R-MMU-114261"  "R-MMU-6804642"
[4849] "R-MMU-9627056" "R-MMU-9627078" "R-MMU-9627089" "R-NUL-992753" 
[4853] "R-MMU-70569"   "R-MMU-418553"  "R-MMU-448744"  "R-MMU-8986972"
[4857] "R-MMU-8986994" "R-MMU-8987015" "R-MMU-8987039" "R-MMU-8987129"
[4861] "R-MMU-8987179" "R-MMU-8987220" "R-MMU-873919"  "R-MMU-873921" 
[4865] "R-MMU-873922"  "R-MMU-873924"  "R-MMU-873926"  "R-MMU-873927" 
[4869] "R-MMU-877269"  "R-MMU-909552"  "R-MMU-9678561" "R-MMU-9734547"
[4873] "R-MMU-70899"   "R-MMU-111437"  "R-MMU-111438"  "R-MM

[5281] "R-MMU-9832204" "R-MMU-2022398" "R-MMU-2022405" "R-MMU-2065355"
[5285] "R-MMU-9614933" "R-MMU-114560"  "R-MMU-210304"  "R-MMU-216050" 
[5289] "R-MMU-2396029" "R-MMU-265423"  "R-MMU-265426"  "R-MMU-265427" 
[5293] "R-MMU-349593"  "R-MMU-349603"  "R-MMU-354077"  "R-MMU-4085087"
[5297] "R-MMU-4086200" "R-MMU-4088218" "R-MMU-432096"  "R-MMU-443905" 
[5301] "R-MMU-443910"  "R-MMU-5218818" "R-MMU-5218830" "R-MMU-5218836"
[5305] "R-MMU-5218851" "R-NUL-1236951" "R-MMU-110376"  "R-MMU-73931"  
[5309] "R-MMU-73932"   "R-MMU-8851352" "R-MMU-2671885" "R-MMU-8863494"
[5313] "R-MMU-9650165" "R-MMU-1168445" "R-MMU-1168459" "R-MMU-1168927"
[5317] "R-MMU-1169192" "R-MMU-1169229" "R-MMU-1362485" "R-MMU-982765" 
[5321] "R-MMU-982768"  "R-MMU-982778"  "R-MMU-982807"  "R-MMU-381091" 
[5325] "R-MMU-381109"  "R-MMU-5658438" "R-MMU-202702"  "R-MMU-210285" 
[5329] "R-MMU-210290"  "R-MMU-210294"  "R-MMU-109470"  "R-MMU-109480" 
[5333] "R-MMU-2990842" "R-MMU-9615901" "R-MMU-9618378" "R-MMU-9634169"
[5337]

[9321] "R-MMU-5651654" "R-MMU-5651657" "R-MMU-8855062" "R-MMU-162683" 
[9325] "R-MMU-2395873" "R-MMU-2395879" "R-MMU-2395965" "R-MMU-444848" 
[9329] "R-MMU-444859"  "R-MMU-8874435" "R-MMU-9012016" "R-MMU-1679589"
[9333] "R-MMU-9679045" "R-MMU-984689"  "R-MMU-8867240" "R-MMU-1299297"
[9337] "R-MMU-3296309" "R-MMU-444661"  "R-MMU-9613570" "R-MMU-444527" 
[9341] "R-MMU-9704297" "R-MMU-162797"  "R-MMU-965067"  "R-NUL-421961" 
[9345] "R-MMU-9634846" "R-MMU-9639576" "R-MMU-9640074" "R-MMU-9769840"
[9349] "R-MMU-8873850" "R-MMU-426560"  "R-MMU-8953037" "R-MMU-162729" 
[9353] "R-MMU-451056"  "R-MMU-2404142" "R-MMU-2404144" "R-MMU-2855259"
[9357] "R-MMU-74843"   "R-MMU-9717444" "R-MMU-5625717" "R-MMU-5625738"
[9361] "R-MMU-73912"   "R-MMU-73920"   "R-MMU-1640164" "R-MMU-9766551"
[9365] "R-MMU-442422"  "R-MMU-9007882" "R-MMU-9007901" "R-MMU-9008052"
[9369] "R-MMU-9008054" "R-MMU-8874364" "R-MMU-8936519" "R-MMU-1482745"
[9373] "R-MMU-1482771" "R-MMU-1482828" "R-MMU-1482847" "R-MMU-1482862"
[9377]

[9781] "R-MMU-191114"  "R-MMU-202717"  "R-MMU-6807229" "R-MMU-9832829"
[9785] "R-MMU-2509827" "R-MMU-2509831" "R-MMU-2509838" "R-MMU-9755030"
[9789] "R-MMU-351222"  "R-MMU-6786245" "R-MMU-6808464" "R-MMU-9724685"
[9793] "R-MMU-5619422" "R-MMU-446195"  "R-MMU-9018868" "R-MMU-9018895"
[9797] "R-MMU-9020271" "R-MMU-9020273" "R-MMU-350869"  "R-MMU-5696004"
[9801] "R-MMU-9670578" "R-MMU-8955010" "R-MMU-71825"   "R-MMU-5696146"
[9805] "R-MMU-6788810" "R-MMU-8944246" "R-MMU-420384"  "R-MMU-6786239"
[9809] "R-MMU-9717434" "R-MMU-5619427" "R-MMU-1605723" "R-MMU-4085029"
[9813] "R-MMU-8878787" "R-MMU-162821"  "R-MMU-5619433" "R-MMU-6799977"
[9817] "R-MMU-6799981" "R-MMU-174916"  "R-MMU-8941701" "R-MMU-904864" 
[9821] "R-MMU-429663"  "R-MMU-8954327" "R-MMU-159179"  "R-MMU-159194" 
[9825] "R-MMU-9756134" "R-MMU-9756156" "R-MMU-9756183" "R-MMU-9632038"
[9829] "R-MMU-9632039" "R-MMU-5695989" "R-MMU-209821"  "R-MMU-191323" 
[9833] "R-MMU-159790"  "R-MMU-9026967" "R-MMU-5619415" "R-MMU-6807826"
[9837]

In [43]:
%%R
rxn_knn_misclass_rate.nls <- list()
rxn_knn_ari.nls <- list()
rxn_knn_ecount.nls <- list()
rxn_pca.nls <- list()
count <- 0

In [44]:
%%R
toi_indices <- seq(1,length(GEO_model_training_tissue_detail.vec))
print(toi_indices)

   [1]    1    2    3    4    5    6    7    8    9   10   11   12   13   14
  [15]   15   16   17   18   19   20   21   22   23   24   25   26   27   28
  [29]   29   30   31   32   33   34   35   36   37   38   39   40   41   42
  [43]   43   44   45   46   47   48   49   50   51   52   53   54   55   56
  [57]   57   58   59   60   61   62   63   64   65   66   67   68   69   70
  [71]   71   72   73   74   75   76   77   78   79   80   81   82   83   84
  [85]   85   86   87   88   89   90   91   92   93   94   95   96   97   98
  [99]   99  100  101  102  103  104  105  106  107  108  109  110  111  112
 [113]  113  114  115  116  117  118  119  120  121  122  123  124  125  126
 [127]  127  128  129  130  131  132  133  134  135  136  137  138  139  140
 [141]  141  142  143  144  145  146  147  148  149  150  151  152  153  154
 [155]  155  156  157  158  159  160  161  162  163  164  165  166  167  168
 [169]  169  170  171  172  173  174  175  176  177  178  179  180  181  182

[1485] 1485 1486 1487 1488 1489 1490 1491 1492 1493 1494 1495 1496 1497 1498
[1499] 1499 1500 1501 1502 1503 1504 1505 1506 1507 1508 1509 1510 1511 1512
[1513] 1513 1514 1515 1516 1517 1518 1519 1520 1521 1522 1523 1524 1525 1526
[1527] 1527 1528 1529 1530 1531 1532 1533 1534 1535 1536 1537 1538 1539 1540
[1541] 1541 1542 1543 1544 1545 1546 1547 1548 1549 1550 1551 1552 1553 1554
[1555] 1555 1556 1557 1558 1559 1560 1561 1562 1563 1564 1565 1566 1567 1568
[1569] 1569 1570 1571 1572 1573 1574 1575 1576 1577 1578 1579 1580 1581 1582
[1583] 1583 1584 1585 1586 1587 1588 1589 1590 1591 1592 1593 1594 1595 1596
[1597] 1597 1598 1599 1600 1601 1602 1603 1604 1605 1606 1607 1608 1609 1610
[1611] 1611 1612 1613 1614 1615 1616 1617 1618 1619 1620 1621 1622 1623 1624
[1625] 1625 1626 1627 1628 1629 1630 1631 1632 1633 1634 1635 1636 1637 1638
[1639] 1639 1640 1641 1642 1643 1644 1645 1646 1647 1648 1649 1650 1651 1652
[1653] 1653 1654 1655 1656 1657 1658 1659 1660 1661 1662 1663 1664 1665 1666

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)




[5545] 5545 5546 5547 5548 5549 5550 5551 5552 5553 5554 5555 5556 5557 5558
[5559] 5559 5560 5561 5562 5563 5564 5565 5566 5567 5568 5569 5570 5571 5572
[5573] 5573 5574 5575 5576 5577 5578 5579 5580 5581 5582 5583 5584 5585 5586
[5587] 5587 5588 5589 5590 5591 5592 5593 5594 5595 5596 5597 5598 5599 5600
[5601] 5601 5602 5603 5604 5605 5606 5607 5608 5609 5610 5611 5612 5613 5614
[5615] 5615 5616 5617 5618 5619 5620 5621 5622 5623 5624 5625 5626 5627 5628
[5629] 5629 5630 5631 5632 5633 5634 5635 5636 5637 5638 5639 5640 5641 5642
[5643] 5643 5644 5645 5646 5647 5648 5649 5650 5651 5652 5653 5654 5655 5656
[5657] 5657 5658 5659 5660 5661 5662 5663 5664 5665 5666 5667 5668 5669 5670
[5671] 5671 5672 5673 5674 5675 5676 5677 5678 5679 5680 5681 5682 5683 5684
[5685] 5685 5686 5687 5688 5689 5690 5691 5692 5693 5694 5695 5696 5697 5698
[5699] 5699 5700 5701 5702 5703 5704 5705 5706 5707 5708 5709 5710 5711 5712
[5713] 5713 5714 5715 5716 5717 5718 5719 5720 5721 5722 5723 5724 5725 572

In [45]:
%%R
GEO_model_training_tissue_detail_vec_tis_of_interest <-
   GEO_model_training_tissue_detail.vec[toi_indices]
print(GEO_model_training_tissue_detail_vec_tis_of_interest)

   [1] "Liver"         "Brain"         "Intestine"     "Intestine"    
   [5] "Liver"         "Lung"          "Intestine"     "Liver"        
   [9] "Brain"         "Muscle"        "Liver"         "Liver"        
  [13] "Liver"         "Liver"         "Liver"         "Liver"        
  [17] "Brain"         "Muscle"        "Liver"         "Liver"        
  [21] "Testes"        "Heart"         "Skin"          "Brain"        
  [25] "Liver"         "Liver"         "Kidney"        "Testes"       
  [29] "Muscle"        "Skin"          "Intestine"     "Uterus"       
  [33] "Uterus"        "Liver"         "Muscle"        "Liver"        
  [37] "Brain"         "Sperm"         "Testes"        "Brain"        
  [41] "Eye"           "Liver"         "Intestine"     "Liver"        
  [45] "Liver"         "Adipose"       "Liver"         "Muscle"       
  [49] "Heart"         "Adipose"       "Lung"          "Eye"          
  [53] "Liver"         "Liver"         "Adipose"       "Adrenal"      
  [57]

 [461] "Brain"         "Ovary"         "Liver"         "Kidney"       
 [465] "Pancreas"      "Adipose"       "Intestine"     "Kidney"       
 [469] "Liver"         "Brain"         "Lung"          "Liver"        
 [473] "Brain"         "Intestine"     "Intestine"     "Liver"        
 [477] "Kidney"        "Adipose"       "Liver"         "Liver"        
 [481] "Liver"         "Muscle"        "Liver"         "Liver"        
 [485] "Liver"         "Muscle"        "Adipose"       "Adipose"      
 [489] "Adipose"       "MOE"           "Liver"         "Liver"        
 [493] "Adipose"       "Olfactory"     "Testes"        "Lung"         
 [497] "Testes"        "Liver"         "Kidney"        "Brain"        
 [501] "Liver"         "Uterus"        "Brain"         "Brain"        
 [505] "Liver"         "Kidney"        "Intestine"     "Liver"        
 [509] "Liver"         "Adipose"       "Liver"         "Intestine"    
 [513] "Liver"         "Brain"         "Liver"         "Adipose"      
 [517]

[3561] "Liver"         "Intestine"     "Brain"         "Testes"       
[3565] "Eye"           "Adipose"       "Liver"         "Testes"       
[3569] "Brain"         "Brain"         "Muscle"        "Muscle"       
[3573] "Liver"         "Liver"         "Adipose"       "Testes"       
[3577] "Intestine"     "Intestine"     "Liver"         "Brain"        
[3581] "Brain"         "Liver"         "Brain"         "Intestine"    
[3585] "Pancreas"      "Lung"          "Brain"         "MOE"          
[3589] "Mammary gland" "Adipose"       "Liver"         "VNO"          
[3593] "Adipose"       "Brain"         "Liver"         "Muscle"       
[3597] "Adipose"       "Kidney"        "Eye"           "Liver"        
[3601] "Brain"         "Brain"         "Brain"         "Intestine"    
[3605] "Adipose"       "Adipose"       "Lung"          "VNO"          
[3609] "Adipose"       "Muscle"        "Liver"         "MOE"          
[3613] "Nerve"         "Pancreas"      "Brain"         "Testes"       
[3617]

[4733] "Brain"         "Muscle"        "Liver"         "Liver"        
[4737] "Intestine"     "Muscle"        "Eye"           "Adipose"      
[4741] "Adipose"       "Liver"         "Kidney"        "Brain"        
[4745] "MOE"           "Liver"         "Lung"          "Brain"        
[4749] "Pancreas"      "Liver"         "Intestine"     "Intestine"    
[4753] "Kidney"        "Liver"         "Brain"         "Adipose"      
[4757] "Intestine"     "Muscle"        "Brain"         "Lung"         
[4761] "Sperm"         "Brain"         "Lung"          "Brain"        
[4765] "Testes"        "Tendon"        "Brain"         "Brain"        
[4769] "Liver"         "Liver"         "Liver"         "Testes"       
[4773] "Liver"         "Kidney"        "Liver"         "Muscle"       
[4777] "Liver"         "Heart"         "Brain"         "Intestine"    
[4781] "Liver"         "Eye"           "Adipose"       "Heart"        
[4785] "Brain"         "Liver"         "Bone"          "Brain"        
[4789]

[5193] "Muscle"        "Aorta"         "Liver"         "Skin"         
[5197] "Brain"         "Intestine"     "Heart"         "Adipose"      
[5201] "Kidney"        "Eye"           "Uterus"        "Liver"        
[5205] "Testes"        "Liver"         "Pancreas"      "Brain"        
[5209] "Liver"         "Brain"         "Adipose"       "Brain"        
[5213] "Brain"         "Adrenal"       "Liver"         "Kidney"       
[5217] "Muscle"        "MOE"           "Ovary"         "Intestine"    
[5221] "Intestine"     "Liver"         "Bone"          "Liver"        
[5225] "Intestine"     "Testes"        "Kidney"        "Intestine"    
[5229] "Brain"         "Adipose"       "Intestine"     "Liver"        
[5233] "Nerve"         "Testes"        "Liver"         "Adipose"      
[5237] "Brain"         "Kidney"        "Pancreas"      "Adipose"      
[5241] "Intestine"     "Liver"         "Liver"         "Brain"        
[5245] "Liver"         "Liver"         "Liver"         "Testes"       
[5249]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [46]:
%%R
vst.count.mtx.tis_of_interest <- vst.count.mtx[, toi_indices]
print(vst.count.mtx.tis_of_interest)

                          V1        V2        V3        V4        V5        V6
ENSMUSG00000000001 15.509601 12.614846 15.519747 14.637851 14.055201 13.449191
ENSMUSG00000000028  8.444423  7.614687 10.420509  9.853538  7.905623  8.286595
ENSMUSG00000000049 20.595034  6.507014  8.077445  7.916957 20.798988  6.163516
ENSMUSG00000000058 10.526176 11.266110  8.766416 11.650305  9.692303 15.196968
ENSMUSG00000000085  9.811438 13.441422 11.548218 12.198293 11.906671 12.810130
ENSMUSG00000000088 16.480554 13.705892 14.766639 14.470708 18.266176 11.941152
ENSMUSG00000000120  9.570220  8.731973  9.042667  9.580347 11.275285  7.829239
ENSMUSG00000000125  4.150743  8.571661 10.636489  4.498000  3.916735  5.114600
ENSMUSG00000000126  7.073358  7.212174  7.601707  7.519440  7.816728 10.184149
ENSMUSG00000000127 11.007867 10.848654 10.921430 10.695660 11.936978 11.026712
ENSMUSG00000000142 10.089224 11.974981 14.516734 11.808310 10.162598 12.502983
ENSMUSG00000000149 14.462024 13.909147 12.471857 12.

ENSMUSG00000000120  8.673431  8.097015  8.534823  7.509739 14.287382  9.950898
ENSMUSG00000000125 10.691884 11.788137  9.107095  8.447594 10.539471  3.795593
ENSMUSG00000000126  7.430531 10.838424  9.501183 11.598594 10.537542  7.411331
ENSMUSG00000000127 10.858311 14.595709 11.648632 10.663082 12.409896 12.113982
ENSMUSG00000000142 11.562482 10.841656 12.306995 10.917292 14.173770 11.560112
ENSMUSG00000000149 14.777295 14.144058 14.153402 14.278215 12.319568 14.565823
ENSMUSG00000000154  5.937770  4.390378  6.871150  6.201851  5.612252 13.509429
ENSMUSG00000000168 14.229454 12.527486 13.048108 13.461763  9.063156 13.037065
ENSMUSG00000000171 13.815136 13.725101 13.049582 13.765868  9.018551 15.051932
                         V43       V44       V45       V46       V47       V48
ENSMUSG00000000001 14.863801 15.551396 15.509004 12.881099 15.083501 12.890131
ENSMUSG00000000028 10.222841  8.291743  9.959162 10.339742  8.465110  8.994179
ENSMUSG00000000049  8.770823 20.313779 19.718144  4.

ENSMUSG00000000168 12.979826 12.311295 13.708313 13.553119 13.405112 14.667263
ENSMUSG00000000171 12.931969 14.804355 16.474336 13.424510 15.704660 15.702935
                         V79       V80       V81       V82       V83       V84
ENSMUSG00000000001 12.582161 12.503299 12.640717 15.584720 12.360585 15.655105
ENSMUSG00000000028  7.948053  9.399664  9.131678  8.711479  9.039420  3.999819
ENSMUSG00000000049  8.154622  8.265488  9.322241 19.485815  7.646564 10.611460
ENSMUSG00000000058 12.562862 13.554195 13.822162 10.170051 13.003290 14.090987
ENSMUSG00000000085 11.816214 13.551970 13.467942 11.023640 12.215778 10.335457
ENSMUSG00000000088 13.873619 18.052360 17.152793 15.238424 16.525556 17.466672
ENSMUSG00000000120  6.316016  7.212498  7.577347 10.595755  8.724740  9.582843
ENSMUSG00000000125  8.244985  3.757652  5.478328  3.866418  5.409435  3.999819
ENSMUSG00000000126 11.836815 12.289930 12.020563  7.005298 11.212685 10.369105
ENSMUSG00000000127 12.502169 11.426187 11.439485 11.

ENSMUSG00000000085 13.377381 11.950843 10.682188  7.102943 11.774731  7.384592
ENSMUSG00000000088 13.748761 17.098014 14.128454 13.834484 17.289886 12.944803
ENSMUSG00000000120  8.850061  8.496343  8.831408  7.477013  9.084893  7.384592
ENSMUSG00000000125  3.836943  3.591444  8.891622 12.960932  3.682271  7.384592
ENSMUSG00000000126  8.656499 10.409242  7.762952  7.648239  8.997953  7.384592
ENSMUSG00000000127 13.637549 11.891914 11.251963 12.333782 10.935601 13.652284
ENSMUSG00000000142 10.697947 10.144484 13.998688 12.951231 11.356770 19.361243
ENSMUSG00000000149 12.747784 15.369464 11.971137 13.568616 14.846621 15.746153
ENSMUSG00000000154  8.593214  7.100619 12.085728  9.844774  8.249869  7.384592
ENSMUSG00000000168 15.018108 16.758112 12.602269 12.613951 17.571564  7.384592
ENSMUSG00000000171 13.386704 16.754294 14.942897 13.933351 17.264501 17.441888
                        V121      V122      V123      V124     V125      V126
ENSMUSG00000000001 14.743404 15.700680 12.025684 15.1

ENSMUSG00000000154 16.730378 10.771123  6.903917 11.683434  5.189046  6.299271
ENSMUSG00000000168 13.494392 14.510928 14.029953 13.278148  3.657871 13.508188
ENSMUSG00000000171 15.765562 12.158344 13.468091 13.747089  9.439626 13.214816
                        V157      V158      V159      V160      V161      V162
ENSMUSG00000000001 13.354350 13.238303 15.007388 11.390851 14.207675 14.211855
ENSMUSG00000000028  9.545080  9.728817  8.844115  9.679811  9.110544  9.520630
ENSMUSG00000000049  6.079521  5.235127  6.067967  7.860403  7.890092  6.740670
ENSMUSG00000000058 11.852679 11.818355 17.602473  8.845261 14.881035 17.624886
ENSMUSG00000000085 11.783427 11.481405 11.641590 14.091963 13.336967 11.509738
ENSMUSG00000000088 13.884197 14.158383 12.969513 12.757089 19.039605 13.464049
ENSMUSG00000000120  8.740918  8.284981  7.030062 10.070258  7.559985  7.797186
ENSMUSG00000000125  5.990732  6.291696  5.413781 10.983673  3.887093

...

ENSMUSG00000000120  8.563435  7.543937  8.318774  7.007777  8.827166  8.214126
ENSMUSG00000000125  8.532228  7.537498  3.671371  8.587989  9.262967  3.995968
ENSMUSG00000000126  8.931613  9.348435  8.884455  9.300577  8.059237 14.144518
ENSMUSG00000000127 10.673194 12.188524 11.171336 13.207584 12.252268  3.995968
ENSMUSG00000000142 11.362005 10.970370 12.241691 12.238031 10.988115  9.680634
ENSMUSG00000000149 14.249606 12.973351 14.345407 13.867018 15.016102 14.775443
ENSMUSG00000000154  6.683035  9.187385  7.651925  5.624842  5.510969  3.995968
ENSMUSG00000000168 13.670455 12.703219 18.752957 14.682654 14.799881 13.346006
ENSMUSG00000000171 13.458930 13.473522 18.133386 12.607307 13.537037 14.256637
                      V5923     V5924     V5925     V5926     V5927     V5928
ENSMUSG00000000001 9.274985 14.469711 12.911531 12.610573 16.737584 13.086380
ENSMUSG00000000028 9.274985  9.121878  8.183231  9.459305  9.448493  9.874646
ENSMUSG00000000049 9.274985 20.456547 15.666295  7.4149

ENSMUSG00000000171 15.788614 16.373536 14.845826 21.256678  8.866320 16.517334
                       V5959     V5960     V5961     V5962     V5963     V5964
ENSMUSG00000000001 14.844729 15.256078 12.676174 13.994595 13.930627 14.979063
ENSMUSG00000000028  8.232621  8.793868 13.525460  8.190406  9.568610  8.166085
ENSMUSG00000000049 10.404534 19.712792 17.158480  6.117958  6.076162 20.318294
ENSMUSG00000000058 14.084525  9.810536  7.309173 12.098967 12.042264 12.280967
ENSMUSG00000000085 10.506401 10.046315 14.564514 11.537698 11.504683 10.624002
ENSMUSG00000000088 15.416603 15.386571 13.456004 14.013757 11.956421 15.083913
ENSMUSG00000000120  7.165386  9.871511 10.258126  8.139296  9.088893 10.123554
ENSMUSG00000000125  3.734525  3.937958 17.571435  3.614699  5.916779  5.983825
ENSMUSG00000000126  8.267753  8.468890  4.102120  8.398464  9.258381  7.813688
ENSMUSG00000000127 10.561529 10.892492 17.568444 10.507832

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 15.653851 15.563136 13.733033
ENSMUSG00000000171 16.531345 13.097446  8.953446 16.201551 16.214601 15.908683
                       V6145     V6146     V6147     V6148     V6149     V6150
ENSMUSG00000000001 15.060397 14.898583 15.083324 15.362237 13.229954 14.380749
ENSMUSG00000000028 12.478437  9.884222  9.483116  8.188177  8.878407 10.116373
ENSMUSG00000000049  6.418884  7.364481 19.882243 20.172278  8.469204  7.313094
ENSMUSG00000000058 10.938350 11.539889  9.826004 10.423153 12.150497 17.701679
ENSMUSG00000000085 12.807894 11.709675 10.719108 11.084535 11.901205 10.862830
ENSMUSG00000000088 12.265486 14.220229 15.464450 15.090114 13.382914 12.950228
ENSMUSG00000000120  8.158718  9.555467  8.525696 10.828864 11.514120  3.839328
ENSMUSG00000000125  8.479683  3.953795  3.663291  3.833848  6.608131  3.839328
ENSMUSG00000000126  6.682301  7.277547  7.542261  8.108540  8.011766 10.161127
ENSMUSG00000000127 12.909059 11.712215 10.570440 10.874076 12.970371 12.659989
ENSMUSG00000000142 12

ENSMUSG00000000088 18.320812 11.794937 12.883991 12.321800 14.338236 13.106982
ENSMUSG00000000120 11.453586 12.442247  7.439486  9.191534  9.395713 12.985358
ENSMUSG00000000125  3.889595  6.735327  3.716541  6.337338  5.265768  9.509841
ENSMUSG00000000126  6.200021  8.068673  9.635926  8.921481  7.676625  6.961862
ENSMUSG00000000127 11.983181 11.762822 11.953903 12.996016 11.355491 12.367347
ENSMUSG00000000142  9.143509 11.948458 10.942205 11.848770 12.470772 11.696861
ENSMUSG00000000149 13.358113 12.764481 14.081593 13.659464 15.000711 13.320412
ENSMUSG00000000154 16.540062  3.722818  6.502334  3.864646  5.340178  9.263673
ENSMUSG00000000168 15.937029 13.379436 13.179568 12.550707 14.513662 12.781874
ENSMUSG00000000171 19.127373 11.790645 13.224631 12.712022 14.355157 13.097670
                       V6187     V6188     V6189     V6190     V6191     V6192
ENSMUSG00000000001 14.879085 12.144884 13.564916 14.385837 12.744001 15.282920
ENSMUSG00000000028  8.560346  8.409413  9.036266 10.

In [47]:
%%R
training_indices <-
   caret::createDataPartition(
      GEO_model_training_tissue_detail_vec_tis_of_interest,
      times = 1,
      p = 1.0, # no data will be held out when set to "1.0"
      list = FALSE
   )
print(training_indices)


[4041,]      4041
[4042,]      4042
[4043,]      4043
[4044,]      4044
[4045,]      4045
[4046,]      4046
[4047,]      4047
[4048,]      4048
[4049,]      4049
[4050,]      4050
[4051,]      4051
[4052,]      4052
[4053,]      4053
[4054,]      4054
[4055,]      4055
[4056,]      4056
[4057,]      4057
[4058,]      4058
[4059,]      4059
[4060,]      4060
[4061,]      4061
[4062,]      4062
[4063,]      4063
[4064,]      4064
[4065,]      4065
[4066,]      4066
[4067,]      4067
[4068,]      4068
[4069,]      4069
[4070,]      4070
[4071,]      4071
[4072,]      4072
[4073,]      4073
[4074,]      4074
[4075,]      4075
[4076,]      4076
[4077,]      4077
[4078,]      4078
[4079,]      4079
[4080,]      4080
[4081,]      4081
[4082,]      4082
[4083,]      4083
[4084,]      4084
[4085,]      4085
[4086,]      4086
[4087,]      4087
[4088,]      4088
[4089,]      4089
[4090,]      4090
[4091,]      4091
[4092,]      4092
[4093,]      4093
[4094,]      4094
[4095,]      4095
[4096,]  

[4658,]      4658
[4659,]      4659
[4660,]      4660
[4661,]      4661
[4662,]      4662
[4663,]      4663
[4664,]      4664
[4665,]      4665
[4666,]      4666
[4667,]      4667
[4668,]      4668
[4669,]      4669
[4670,]      4670
[4671,]      4671
[4672,]      4672
[4673,]      4673
[4674,]      4674
[4675,]      4675
[4676,]      4676
[4677,]      4677
[4678,]      4678
[4679,]      4679
[4680,]      4680
[4681,]      4681
[4682,]      4682
[4683,]      4683
[4684,]      4684
[4685,]      4685
[4686,]      4686
[4687,]      4687
[4688,]      4688
[4689,]      4689
[4690,]      4690
[4691,]      4691
[4692,]      4692
[4693,]      4693
[4694,]      4694
[4695,]      4695
[4696,]      4696
[4697,]      4697
[4698,]      4698
[4699,]      4699
[4700,]      4700
[4701,]      4701
[4702,]      4702
[4703,]      4703
[4704,]      4704
[4705,]      4705
[4706,]      4706
[4707,]      4707
[4708,]      4708
[4709,]      4709
[4710,]      4710
[4711,]      4711
[4712,]      4712
[4713,]   

[5113,]      5113
[5114,]      5114
[5115,]      5115
[5116,]      5116
[5117,]      5117
[5118,]      5118
[5119,]      5119
[5120,]      5120
[5121,]      5121

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [48]:
%%R
vst.count.mtx.train <-
   vst.count.mtx.tis_of_interest[, training_indices] #9/10ths of data
vst.count.mtx.test  <-
   vst.count.mtx.tis_of_interest[, -training_indices] #1/10th of data

print(vst.count.mtx.train)
print(length(vst.count.mtx.test))

                          V1        V2        V3        V4        V5        V6
ENSMUSG00000000001 15.509601 12.614846 15.519747 14.637851 14.055201 13.449191
ENSMUSG00000000028  8.444423  7.614687 10.420509  9.853538  7.905623  8.286595
ENSMUSG00000000049 20.595034  6.507014  8.077445  7.916957 20.798988  6.163516
ENSMUSG00000000058 10.526176 11.266110  8.766416 11.650305  9.692303 15.196968
ENSMUSG00000000085  9.811438 13.441422 11.548218 12.198293 11.906671 12.810130
ENSMUSG00000000088 16.480554 13.705892 14.766639 14.470708 18.266176 11.941152
ENSMUSG00000000120  9.570220  8.731973  9.042667  9.580347 11.275285  7.829239
ENSMUSG00000000125  4.150743  8.571661 10.636489  4.498000  3.916735  5.114600
ENSMUSG00000000126  7.073358  7.212174  7.601707  7.519440  7.816728 10.184149
ENSMUSG00000000127 11.007867 10.848654 10.921430 10.695660 11.936978 11.026712
ENSMUSG00000000142 10.089224 11.974981 14.516734 11.808310 10.162598 12.502983
ENSMUSG00000000149 14.462024 13.909147 12.471857 12.

ENSMUSG00000000120  8.673431  8.097015  8.534823  7.509739 14.287382  9.950898
ENSMUSG00000000125 10.691884 11.788137  9.107095  8.447594 10.539471  3.795593
ENSMUSG00000000126  7.430531 10.838424  9.501183 11.598594 10.537542  7.411331
ENSMUSG00000000127 10.858311 14.595709 11.648632 10.663082 12.409896 12.113982
ENSMUSG00000000142 11.562482 10.841656 12.306995 10.917292 14.173770 11.560112
ENSMUSG00000000149 14.777295 14.144058 14.153402 14.278215 12.319568 14.565823
ENSMUSG00000000154  5.937770  4.390378  6.871150  6.201851  5.612252 13.509429
ENSMUSG00000000168 14.229454 12.527486 13.048108 13.461763  9.063156 13.037065
ENSMUSG00000000171 13.815136 13.725101 13.049582 13.765868  9.018551 15.051932
                         V43       V44       V45       V46       V47       V48
ENSMUSG00000000001 14.863801 15.551396 15.509004 12.881099 15.083501 12.890131
ENSMUSG00000000028 10.222841  8.291743  9.959162 10.339742  8.465110  8.994179
ENSMUSG00000000049  8.770823 20.313779 19.718144  4.

ENSMUSG00000000168 12.979826 12.311295 13.708313 13.553119 13.405112 14.667263
ENSMUSG00000000171 12.931969 14.804355 16.474336 13.424510 15.704660 15.702935
                         V79       V80       V81       V82       V83       V84
ENSMUSG00000000001 12.582161 12.503299 12.640717 15.584720 12.360585 15.655105
ENSMUSG00000000028  7.948053  9.399664  9.131678  8.711479  9.039420  3.999819
ENSMUSG00000000049  8.154622  8.265488  9.322241 19.485815  7.646564 10.611460
ENSMUSG00000000058 12.562862 13.554195 13.822162 10.170051 13.003290 14.090987
ENSMUSG00000000085 11.816214 13.551970 13.467942 11.023640 12.215778 10.335457
ENSMUSG00000000088 13.873619 18.052360 17.152793 15.238424 16.525556 17.466672
ENSMUSG00000000120  6.316016  7.212498  7.577347 10.595755  8.724740  9.582843
ENSMUSG00000000125  8.244985  3.757652  5.478328  3.866418  5.409435  3.999819
ENSMUSG00000000126 11.836815 12.289930 12.020563  7.005298 11.212685 10.369105
ENSMUSG00000000127 12.502169 11.426187 11.439485 11.

ENSMUSG00000000085 13.377381 11.950843 10.682188  7.102943 11.774731  7.384592
ENSMUSG00000000088 13.748761 17.098014 14.128454 13.834484 17.289886 12.944803
ENSMUSG00000000120  8.850061  8.496343  8.831408  7.477013  9.084893  7.384592
ENSMUSG00000000125  3.836943  3.591444  8.891622 12.960932  3.682271  7.384592
ENSMUSG00000000126  8.656499 10.409242  7.762952  7.648239  8.997953  7.384592
ENSMUSG00000000127 13.637549 11.891914 11.251963 12.333782 10.935601 13.652284
ENSMUSG00000000142 10.697947 10.144484 13.998688 12.951231 11.356770 19.361243
ENSMUSG00000000149 12.747784 15.369464 11.971137 13.568616 14.846621 15.746153
ENSMUSG00000000154  8.593214  7.100619 12.085728  9.844774  8.249869  7.384592
ENSMUSG00000000168 15.018108 16.758112 12.602269 12.613951 17.571564  7.384592
ENSMUSG00000000171 13.386704 16.754294 14.942897 13.933351 17.264501 17.441888
                        V121      V122      V123      V124     V125      V126
ENSMUSG00000000001 14.743404 15.700680 12.025684 15.1

ENSMUSG00000000154 16.730378 10.771123  6.903917 11.683434  5.189046  6.299271
ENSMUSG00000000168 13.494392 14.510928 14.029953 13.278148  3.657871 13.508188
ENSMUSG00000000171 15.765562 12.158344 13.468091 13.747089  9.439626 13.214816
                        V157      V158      V159      V160      V161      V162
ENSMUSG00000000001 13.354350 13.238303 15.007388 11.390851 14.207675 14.211855
ENSMUSG00000000028  9.545080  9.728817  8.844115  9.679811  9.110544  9.520630
ENSMUSG00000000049  6.079521  5.235127  6.067967  7.860403  7.890092  6.740670
ENSMUSG00000000058 11.852679 11.818355 17.602473  8.845261 14.881035 17.624886
ENSMUSG00000000085 11.783427 11.481405 11.641590 14.091963 13.336967 11.509738
ENSMUSG00000000088 13.884197 14.158383 12.969513 12.757089 19.039605 13.464049
ENSMUSG00000000120  8.740918  8.284981  7.030062 10.070258  7.559985  7.797186
ENSMUSG00000000125  5.990732  6.291696  5.413781 10.983673  3.887093

...

ENSMUSG00000000058 10.235046 10.625145  3.886412  9.621871  9.974506  9.693193
ENSMUSG00000000085 10.138069  9.901474 12.449888 11.447157 10.577170 11.326117
ENSMUSG00000000088 15.642191 12.620428 13.727800 14.962708 15.528956 15.437935
ENSMUSG00000000120  6.770248  5.747965  8.080874 10.588896  9.574366 10.123149
ENSMUSG00000000125  4.117762  8.447962  5.292788  3.648555  4.323920  3.878349
ENSMUSG00000000126  7.357851  7.372180  8.121133  7.992632  6.855519  6.447792
ENSMUSG00000000127 10.500306 12.656729 11.390361 10.952762 10.987985 10.697609
ENSMUSG00000000142  9.780042 13.877760 10.884148 11.912408 10.307309 10.797907
ENSMUSG00000000149 14.586409 12.606008  3.886412 15.235128 16.238395 16.166659
ENSMUSG00000000154 14.059397  9.820690  3.886412 14.738591 14.912850 15.095699
ENSMUSG00000000168 13.915679 11.448160 13.050074 14.541797 13.472013 13.217468
ENSMUSG00000000171 16.692570 12.688664 13.668540 16.243413 15.952947 16.357924
                       V6169     V6170     V6171    

ENSMUSG00000000142 10.461203 12.065099 14.979448 10.19056 10.401959 10.865336
ENSMUSG00000000149 14.338105 13.829819  5.222261 10.19056 14.511195 16.009066
ENSMUSG00000000154 14.499943  3.967361  3.604600 10.19056  9.173716 15.529470
ENSMUSG00000000168 13.026291 13.032765  7.078368 10.19056 14.252776 13.749705
ENSMUSG00000000171 15.542953 13.352303  8.617884 10.19056 14.260668 16.419617
                       V6205     V6206     V6207     V6208     V6209     V6210
ENSMUSG00000000001 13.926609 15.286455 14.858058 16.321015 12.818054 14.552353
ENSMUSG00000000028  8.998597  8.306705  8.614623 10.352468  9.700233  7.512401
ENSMUSG00000000049 11.862057 20.448007  3.674258  8.960787 15.718235  9.044020
ENSMUSG00000000058 12.711278  9.603395 16.929042 10.086459  9.275558 11.989329
ENSMUSG00000000085 12.298985 11.530821 12.198121 10.533775 11.999062 12.432327
ENSMUSG00000000088 12.594777 15.422305 14.704692 14.521650 11.882866 15.281375
ENSMUSG00000000120 10.914850 10.650670  7.458856  8.28430

ENSMUSG00000000028  8.505749  8.588199  9.251577  8.014782 10.056184  8.292622
ENSMUSG00000000049  8.458582 20.604534  7.136453 19.686528  6.020338 19.181866
ENSMUSG00000000058 11.555125  8.892526 10.215526  9.862672 17.281617 10.628684
ENSMUSG00000000085 12.214052 11.296480 12.768462 10.154968 11.710716 10.558919
ENSMUSG00000000088 15.866786 16.340492 12.812393 15.464538 13.095366 14.971226
ENSMUSG00000000120  7.832388  8.381543 10.315705  9.639437 11.241533  9.877892
ENSMUSG00000000125  4.982452  3.752502  8.142785  3.936710 13.356679  3.850860
ENSMUSG00000000126  7.832388  7.771640  9.625873  8.523730  6.916858  7.103272
ENSMUSG00000000127 10.468161 10.236158 12.584763 10.999639 11.817230 11.427535
ENSMUSG00000000142  7.832388 10.544538 10.863582 10.773222 11.959959  9.922060
ENSMUSG00000000149 13.856342 16.444720 12.352723 15.888275  9.773439 14.976101
ENSMUSG00000000154 15.977517 15.369053  8.211286 14.641970 10.164534 14.586601
ENSMUSG00000000168 15.373093 13.800960 13.197817 13.

ENSMUSG00000000126  9.918267  8.964152 10.027974 10.942206 10.640213  9.739523
ENSMUSG00000000127  9.918267  3.918617 12.246328 12.525465 15.454857  9.604726
ENSMUSG00000000142  9.918267 12.026957  9.339170 11.277853  8.014199 12.355119
ENSMUSG00000000149 16.652271 19.174996 14.999482 12.974548 14.793443 11.571978
ENSMUSG00000000154  9.918267  8.255351  6.421235  7.239064  6.467894  8.081604
ENSMUSG00000000168  9.918267 15.176198 17.494982 16.016376 10.297827 10.710487
ENSMUSG00000000171 25.111021 16.349947 17.336694 16.187061 11.202554  7.568054
                       V6283     V6284     V6285     V6286     V6287     V6288
ENSMUSG00000000001 12.550480  9.388398 14.170566 14.123262 15.268152 14.250009
ENSMUSG00000000028  9.428608  9.388398  8.568609  9.392270 10.577353 12.536177
ENSMUSG00000000049  7.431090 17.957548  6.051893  5.871877  6.785904 12.203622
ENSMUSG00000000058  3.876791  9.388398 16.873348

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [49]:
%%R
GEO_model_training_tissue_detail.vec.train <-
   GEO_model_training_tissue_detail_vec_tis_of_interest[training_indices]
GEO_model_training_tissue_detail.vec.test <-
   GEO_model_training_tissue_detail_vec_tis_of_interest[-training_indices]

# print(GEO_model_training_tissue_detail.vec.train)
print(length(unique(GEO_model_training_tissue_detail.vec.train)))

[1] 26


In [50]:
%%R
saveRDS(GEO_model_training_tissue_detail.vec.train,file=paste(OUT_DIR,"GEO_model_training_tissue_detail_vec_train.Rds",sep=""))

In [51]:
%%R
cv_fold_indices <- caret::createFolds(GEO_model_training_tissue_detail.vec.train,
                                      k = N_FOLDS)
print(cv_fold_indices)

$Fold01
  [1]    9   10   42   58   64   72   76   90   92  104  105  113  123  137  163
 [16]  177  183  186  203  219  244  249  269  300  306  311  331  340  348  362
 [31]  366  368  370  371  374  380  389  418  435  447  454  473  488  516  519
 [46]  522  529  531  542  573  574  579  585  612  617  628  632  653  658  664
 [61]  674  675  677  687  708  711  712  714  765  791  794  799  816  823  824
 [76]  831  834  837  847  856  874  876  886  889  911  921  925  929  930  934
 [91]  944  960  969  993 1002 1021 1032 1034 1041 1047 1050 1053 1064 1070 1075
[106] 1090 1106 1129 1131 1133 1144 1152 1153 1171 1176 1188 1218 1226 1228 1231
[121] 1268 1271 1276 1290 1306 1320 1354 1362 1371 1397 1405 1417 1424 1425 1427
[136] 1430 1439 1473 1489 1497 1522 1532 1538 1540 1542 1549 1564 1569 1579 1580
[151] 1584 1594 1600 1607 1624 1638 1653 1656 1679 1683 1715 1729 1734 1765 1767
[166] 1769 1779 1783 1789 1792 1794 1801 1821 1822 1826 1833 1844 1846 1859 1866
[181] 1869 1870 1875

[481] 4787 4794 4807 4849 4858 4864 4865 4870 4882 4883 4903 4912 4918 4924 4940
[496] 4960 4965 4966 4983 4986 4988 4991 4993 5022 5025 5029 5037 5042 5078 5089
[511] 5098 5106 5124 5128 5137 5141 5150 5164 5168 5198 5204 5212 5216 5217 5227
[526] 5239 5261 5268 5290 5321 5323 5347 5349 5352 5366 5375 5376 5377 5391 5412
[541] 5414 5417 5424 5427 5428 5449 5454 5456 5462 5466 5488 5508 5521 5533 5546
[556] 5558 5565 5567 5590 5591 5614 5622 5627 5631 5641 5647 5664 5668 5671 5676
[571] 5683 5686

In [52]:
%%R
binary_GEO_model_training_tissue_annotations <- unique(GEO_model_training_tissue_detail.vec)
print(binary_GEO_model_training_tissue_annotations)

In [53]:
%%R
full_rxn_pca_results.nls <- list()
rxn_id_2_result_file_idx.nls <- list()

In [54]:
%%R
print(vst.count.mtx.train[rxn2ensembls.nls[["R-MMU-450466"]], ])

                         V1       V2       V3       V4       V5       V6
ENSMUSG00000000568 13.30169 15.56688 14.58060 13.76051 14.58367 14.63848
ENSMUSG00000002102 15.80848 13.72309 14.45077 13.84377 15.96145 13.97942
ENSMUSG00000005625 14.93809 13.37089 13.59246 13.16788 16.64793 13.28656
ENSMUSG00000005779 16.24652 13.35461 14.07438 13.32048 17.52613 13.15359
ENSMUSG00000006998 15.54118 14.81009 14.38071 14.22403 16.31466 14.01501
ENSMUSG00000014769 16.29169 13.84514 14.50487 14.01661 16.85746 13.18125
ENSMUSG00000015671 15.27505 13.22060 13.37289 12.90380 17.12415 12.81868
ENSMUSG00000017221 14.34028 13.97938 15.01994 13.21428 16.02733 13.60018
ENSMUSG00000017428 14.58578 12.86756 13.80023 12.81980 17.26581 13.17938
ENSMUSG00000018286 15.71306 12.87155 13.68761 13.11953 16.12952 13.34629
ENSMUSG00000020708 14.56493 13.92394 13.96445 13.11676 14.42999 13.18758
ENSMUSG00000020720 15.08808 13.26518 13.23263 12.76769 14.88984 12.53930
ENSMUSG00000021024 16.33752 13.35242 13.89908 13.31

ENSMUSG00000021737 12.98054 13.17005 14.01626 14.03036 16.31468 13.66408
                        V43      V44      V45      V46      V47      V48
ENSMUSG00000000568 13.47371 12.34403 12.93931 13.94653 14.02195 12.92395
ENSMUSG00000002102 13.38190 14.90539 15.15211 14.81704 16.05357 15.90422
ENSMUSG00000005625 12.77783 14.64428 14.59510 14.96003 16.33874 14.30280
ENSMUSG00000005779 13.55801 14.89115 15.37979 13.99381 16.37441 15.16465
ENSMUSG00000006998 14.26853 15.67597 16.13843 15.07051 16.73912 15.45260
ENSMUSG00000014769 13.34639 15.02741 15.25469 14.72672 16.01460 14.80611
ENSMUSG00000015671 12.62518 14.20786 14.21523 14.22465 14.65995 13.80451
ENSMUSG00000017221 13.28810 13.72500 14.96969 14.67674 15.87302 14.42715
ENSMUSG00000017428 12.67447 14.28151 15.18089 14.32790 15.33926 13.30488
ENSMUSG00000018286 12.78357 14.77113 15.02740 14.74701 15.83475 14.90946
ENSMUSG00000020708 13.02142 14.04673 14.43717 13.90718 15.25008 14.51017
ENSMUSG00000020720 12.09449 14.40886 14.63440 14.36

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 15.12676 14.73166 15.44188 15.54028 13.98811
ENSMUSG00000014769 14.65826 14.34040 13.65739 14.01505 14.30227 14.06629
ENSMUSG00000015671 13.64944 12.54546 13.08146 13.87130 14.06870 12.98848
ENSMUSG00000017221 13.95356 15.43815 13.69547 12.97711 13.07253 13.64124
ENSMUSG00000017428 13.59842 14.46780 12.76084 12.11065 11.09812 12.94669
ENSMUSG00000018286 14.71709 14.56484 13.25037 13.69774 14.04057 13.62403
ENSMUSG00000020708 12.75903 14.39068 12.97124 14.28420 13.85575 13.12778
ENSMUSG00000020720 13.98910 14.15293 12.58289 13.66305 13.29530 13.34168
ENSMUSG00000021024 14.06835 14.14584 13.35495 13.58029 12.97014 12.96454
ENSMUSG00000021178 14.32687 14.74832 14.04424 14.54267 14.37329 13.26251
ENSMUSG00000021737 13.44912 13.37778 12.45235 13.85555 13.82351 15.35682
                       V253     V254     V255     V256     V257     V258
ENSMUSG00000000568 13.43287 12.19527 13.57676 13.11835 12.29905 13.04192
ENSMUSG00000002102 15.23660 15.01182 15.11974 13.93946 14.93646 11.79862
ENSMU

ENSMUSG00000006998 13.19432 13.01901 15.67321 16.16090 14.84007 14.77779
ENSMUSG00000014769 13.49189 13.07138 14.28458 15.40753 13.71775 13.94944
ENSMUSG00000015671 13.73194 11.21793 14.25214 14.51428 13.28031 13.04932
ENSMUSG00000017221 13.11526 13.32117 14.90979 15.04826 14.79694 12.92916
ENSMUSG00000017428 13.70451 14.52812 13.42472 14.65676 13.58612 12.85872
ENSMUSG00000018286 13.88605 10.96216 14.08083 15.11107 13.96862 13.30082
ENSMUSG00000020708 13.63530 10.49620 14.78801 14.76458 13.87185 13.03699
ENSMUSG00000020720 14.02460 12.07317 13.49084 15.06631 13.76742 12.39443
ENSMUSG00000021024 14.54633 10.11343 13.90552 15.55716 13.82265 13.29068
ENSMUSG00000021178 13.38955 11.53916 14.47340 15.18480 13.92051 13.82832
ENSMUSG00000021737 13.52097 12.08993 13.72353 14.80528 13.71347 12.74580
                       V295     V296     V297     V298     V299     V300
ENSMUSG00000000568 15.36094 11.91586 15.22935 13.00954 14.32229 15.17622
ENSMUSG00000002102 13.37732 15.33351 14.02934 15.03

...

 22.84290 14.97082  9.882425 16.04199 15.14475 14.36612
ENSMUSG00000014769 15.50298 14.04585  9.882425 15.37826 14.20588 14.22917
ENSMUSG00000015671 16.74331 12.92010 19.062275 14.22742 13.39440 13.04898
ENSMUSG00000017221  8.60580 14.36792  9.882425 15.04562 13.27152 14.48653
ENSMUSG00000017428  8.60580 13.54253  9.882425 14.10574 13.01216 12.69068
ENSMUSG00000018286  8.60580 14.06010  9.882425 14.65669 13.85652 13.41368
ENSMUSG00000020708  8.60580 12.94138  9.882425 13.90663 13.30786 13.50756
ENSMUSG00000020720 19.11635 13.30960 16.605372 14.48653 13.15431 13.13998
ENSMUSG00000021024  8.60580 13.00174 17.808990 15.29860 13.83262 13.20082
ENSMUSG00000021178  8.60580 13.94277  9.882425 15.29408 14.06624 13.53882
ENSMUSG00000021737  8.60580 13.25101  9.882425 14.75515 13.16653 13.51301
                      V5822    V5823     V5824    V5825    V5826    V5827
ENSMUSG00000000568 15.81944 11.92668  9.952191 12.46014 13.64137 13.44068
ENSMUSG00000002102 18.72415 14.81515  9.952191 13.93485 

ENSMUSG00000005779 14.38188 14.35339 26.465131 15.042938 16.15525 15.40180
ENSMUSG00000006998 14.94779 13.73257  9.491167 16.504716 14.67832 15.91239
ENSMUSG00000014769 14.22368 14.03239  9.491167 14.947755 16.34991 14.67079
ENSMUSG00000015671 13.84734 14.16725 15.952238 16.359175 15.57975 14.09809
ENSMUSG00000017221 13.50290 11.59354  9.491167 14.370468 14.01047 14.77181
ENSMUSG00000017428 12.93038 12.68974 22.026249 14.543962 14.14537 14.35411
ENSMUSG00000018286 13.54678 12.39982  9.491167 15.553494 16.19561 14.30430
ENSMUSG00000020708 13.50122 13.76278  9.491167 15.630497 14.28823 13.91774
ENSMUSG00000020720 12.88809 13.20239  9.491167 15.092725 14.18050 14.49910
ENSMUSG00000021024 13.62413 15.00272  9.491167 16.330704 15.29553 14.78156
ENSMUSG00000021178 13.74638 12.88711  9.491167 15.886503 14.42468 15.13725
ENSMUSG00000021737 13.43674 13.36631  9.491167 15.247461 14.95031 14.01546
                      V5864    V5865    V5866    V5867    V5868    V5869
ENSMUSG00000000568 13.45076

ENSMUSG00000005625 13.89356 14.89889 16.04170  8.050469 13.12799 15.236561
ENSMUSG00000005779 14.40888 14.25452 15.05463  8.790957 12.17160 14.167697
ENSMUSG00000006998 15.41666 15.50785 16.83274 20.028720 14.14138 17.708127
ENSMUSG00000014769 14.30593 14.25805 14.62811 12.403280 12.14447 14.614568
ENSMUSG00000015671 13.53801 13.45897 14.00634 16.877030 11.93984 15.783397
ENSMUSG00000017221 14.73260 14.43482 15.64239 18.379089 13.35221 16.332933
ENSMUSG00000017428 13.03063 13.75021 15.19123 16.683823 12.75805 15.884218
ENSMUSG00000018286 13.52038 14.24225 15.73556 17.041419 12.69308 16.651846
ENSMUSG00000020708 14.70077 13.48281 15.83374 17.017883 12.54271 16.450117
ENSMUSG00000020720 13.32559 13.61130 15.48229 16.889969 12.35936 16.039690
ENSMUSG00000021024 13.59179 14.77480 14.76145 17.240484 12.62464 16.077610
ENSMUSG00000021178 14.39066 14.67977 15.57083 17.377655 12.67222 15.783299
ENSMUSG00000021737 13.52737 13.07899 15.59925 16.770127 11.90501 16.205569
                      V59

ENSMUSG00000021178 13.68487 16.77666 15.84026 14.07842 15.38648 16.628856
ENSMUSG00000021737 12.54985 15.64964 15.12969 14.49557 14.21367 16.390755
                       V6236    V6237     V6238    V6239    V6240    V6241
ENSMUSG00000000568  7.139468 12.54610 24.587049 14.65050 13.41390 12.48007
ENSMUSG00000002102 15.547049 14.17300  9.351352 15.19672 13.84048 13.62904
ENSMUSG00000005625 15.140841 14.20734 25.596645 14.43276 14.44636 13.09377
ENSMUSG00000005779 15.412128 13.83515 24.880107 15.60597 14.05821 14.03948
ENSMUSG00000006998 16.287004 15.27233  9.351352 15.85223 14.88839 12.61641
ENSMUSG00000014769 15.714337 14.00762 23.928642 15.36353 14.30158 13.53784
ENSMUSG00000015671 14.716852 13.46181 25.689313 14.20603 13.58893 14.01815
ENSMUSG00000017221 15.288675 13.54588  9.351352 13.63199 14.21501 12.42514
ENSMUSG00000017428 13.856225 13.09144  9.351352 13.61879 13.59436 13.66862
ENSMUSG00000018286 14.903720 13.78519 17.717449 14.80422 13.52351 14.00379
ENSMUSG00000020708 15.26191

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [55]:
%%R
test <- prcomp(t(vst.count.mtx.train[rxn2ensembls.nls[["R-MMU-450466"]], ]))
print(test)

Standard deviations (1, .., p=45):
 [1] 6.1565273 3.7747365 3.5238278 3.0996064 2.4582687 2.3861184 2.1698846
 [8] 2.0092754 1.8921873 1.8610500 1.8160279 1.7558833 1.6286458 1.6037730
[15] 1.5012517 1.4505438 1.4297441 1.3981732 1.3535975 1.2987999 1.2737774
[22] 1.2338582 1.2264620 1.1937135 1.1540139 1.1511725 1.1248728 1.1014481
[29] 1.0794738 1.0709732 1.0394871 1.0189938 1.0018784 0.9613699 0.9576238
[36] 0.9312078 0.9180214 0.8962513 0.8890558 0.8441747 0.8070428 0.7798933
[43] 0.7398722 0.7277171 0.5435962

Rotation (n x k) = (45 x 45):
                           PC1          PC2          PC3         PC4
ENSMUSG00000000568  0.01312248  0.083666994  0.161354758 -0.28822781
ENSMUSG00000002102 -0.15991210  0.185907704  0.263264391  0.14955432
ENSMUSG00000005625 -0.17245049  0.143793396  0.261492377  0.12338099
ENSMUSG00000005779 -0.19031528  0.073036087  0.236897585  0.12877865
ENSMUSG00000006998 -0.14495380  0.101066722 -0.146315145  0.07756111
ENSMUSG00000014769 -0.17318823 -0.0

ENSMUSG00000021737 -0.019989593 -0.024262736  0.0109774197  0.0387762742
ENSMUSG00000021832 -0.116827059 -0.087678063  0.0633533158  0.0666718155
ENSMUSG00000022193  0.194129159  0.053827820 -0.0614425706 -0.1049412237
ENSMUSG00000022216 -0.096515062  0.095313012 -0.1351969133 -0.0535329531
ENSMUSG00000024338  0.152769411 -0.365684276  0.1099914314 -0.0627512462
ENSMUSG00000025487  0.233851560  0.030949547  0.4992706919 -0.0408329870
ENSMUSG00000026229 -0.061640659 -0.142962545  0.0258546862 -0.0807854618
ENSMUSG00000026750 -0.243609610 -0.198824426 -0.0309720115  0.0896055673
ENSMUSG00000026869 -0.146710826 -0.424802407 -0.0736409251 -0.0539844110
ENSMUSG00000026914 -0.235166686 -0.193211795  0.0514400219  0.0283361444
ENSMUSG00000027566 -0.098055078  0.112966613  0.1499993427 -0.1320342385
ENSMUSG00000028837  0.049464575 -0.037915360 -0.0027670443  0.4276592943
ENSMUSG00000028932  0.123302764 -0.022609908 -0.1382127471  0.4913093395
ENSMUSG00000029440  0.047982834 -0.027170687 -0.226

ENSMUSG00000036743  0.007674375  0.032734383 -0.069561969 -0.040506648
ENSMUSG00000039067  0.411238923 -0.290800849  0.329302833 -0.158404066
ENSMUSG00000040850 -0.128743744 -0.009098634  0.037709699 -0.008104902
ENSMUSG00000060073 -0.118553753  0.007610073 -0.076620652 -0.133415912
ENSMUSG00000068749 -0.146111295  0.039076508  0.119420839 -0.040936017
ENSMUSG00000069744  0.077766420  0.026480681 -0.005633563  0.170144228
ENSMUSG00000072423 -0.027001773  0.054689390  0.124470408  0.081148932
ENSMUSG00000078153  0.099549271 -0.033376207 -0.004188799 -0.074459870
ENSMUSG00000078652  0.010592428  0.016344006 -0.397207961 -0.114148423
ENSMUSG00000079197  0.107222446 -0.010135923  0.041461142 -0.036310276
                           PC21         PC22          PC23         PC24
ENSMUSG00000000568 -0.105570232  0.222271972  0.0520935936  0.171967840
ENSMUSG00000002102  0.158317207 -0.008932704 -0.0121120643  0.180881676
ENSMUSG00000005625 -0.106407490  0.107457951 -0.3595495605 -0.166697153
EN

ENSMUSG00000020708 -0.003369031 -0.09213092  0.0871479602  0.120383795
ENSMUSG00000020720  0.057219572 -0.15730322  0.0010423791 -0.239974755
ENSMUSG00000021024  0.115918817  0.21912355 -0.0151801435  0.195310476
ENSMUSG00000021178 -0.170743853  0.05364523  0.0140319427 -0.025494098
ENSMUSG00000021737 -0.139783411  0.07242402 -0.0669878713  0.054140460
ENSMUSG00000021832 -0.112696723  0.07324685 -0.0961233414  0.110098252
ENSMUSG00000022193  0.136501221 -0.15647081 -0.0574223665  0.083819959
ENSMUSG00000022216  0.313588366  0.35365043  0.0567157590 -0.100755459
ENSMUSG00000024338 -0.026843537 -0.02482372 -0.0047511349  0.009854629
ENSMUSG00000025487 -0.101512666 -0.02639676  0.2328600342 -0.166472427
ENSMUSG00000026229 -0.074460786 -0.41353501 -0.1082322423  0.168366844
ENSMUSG00000026750 -0.123288207  0.07746992 -0.3935457314 -0.099858037
ENSMUSG00000026869  0.149215161  0.01098217  0.0327514592 -0.069978913
ENSMUSG00000026914  0.017902814  0.01543445  0.2891437894  0.204144851
ENSMUS

ENSMUSG00000031429 -0.045302105  0.082987612 -3.558957e-02  0.053394143
ENSMUSG00000031897  0.001840587  0.016518887  2.563382e-02 -0.021571825
ENSMUSG00000032301  0.082358222  0.013230207  1.232354e-02  0.025458055
ENSMUSG00000032869  0.313023509  0.086396694 -3.457087e-02 -0.069780256
ENSMUSG00000036743 -0.017931193  0.082905732  3.151531e-02  0.013292363
ENSMUSG00000039067 -0.029061180 -0.047384044 -4.103544e-02 -0.040154846
ENSMUSG00000040850  0.006636855 -0.105318646  2.785689e-02  0.110118130
ENSMUSG00000060073  0.203540313  0.014789144 -3.305193e-02  0.251327683
ENSMUSG00000068749 -0.271740249  0.118556346 -5.784110e-03 -0.258500675
ENSMUSG00000069744 -0.037318293  0.170754812  4.324125e-01  0.007971486
ENSMUSG00000072423  0.126708637 -0.064650539 -2.081272e-01 -0.018740642
ENSMUSG00000078153  0.172895056 -0.050606880 -6.444856e-02  0.197758169
ENSMUSG00000078652 -0.033989625 -0.338014283  1.840020e-01 -0.137837858
ENSMUSG00000079197  0.084996833 -0.059454371 -1.015139e-01  0.17

In [56]:
%%R
test$x[,1]

           V1            V2            V3            V4            V5 
 -8.993708673   5.103485547  -0.312732373   4.469726309 -12.069119001 
           V6            V7            V8            V9           V10 
  4.388113431  -2.923769218   1.454567147  21.635864951  -2.652278230 
          V11           V12           V13           V14           V15 
  6.664965398   1.174731831  -9.415778469  -9.030742235  -8.510689628 
          V16           V17           V18           V19           V20 
-11.287592558  12.185520811  -7.443892276  -2.085871345  -5.993107172 
          V21           V22           V23           V24           V25 
  4.513428263   2.026303852 -12.306215636  16.651960085  -6.391425646 
          V26           V27           V28           V29           V30 
 -5.198447493  -0.389640746   3.972720419  -4.283916724   0.225914397 
          V31           V32           V33           V34           V35 
  1.092841611   1.108450646   8.449001486  -6.201791792  -7.360273457 
      

 -1.924424190   0.166414624  -6.161529281   2.164938256  12.158553317 
         V291          V292          V293          V294          V295 
  0.418569306  -6.884814838   1.411546263   4.017295459   7.882254751 
         V296          V297          V298          V299          V300 
 -1.370957558   4.254637992   0.140162201   4.813441178   4.053833417 
         V301          V302          V303          V304          V305 
 11.459800705   3.771334598  -2.883635830 -19.282796680  -0.278455370 
         V306          V307          V308          V309          V310 
  1.709693030 -10.257166464  -5.454847356  -2.894491787  -1.977733751 
         V311          V312          V313          V314          V315 
  3.964791035   1.628524820   3.017573845 -10.747038577  -1.357604875 
         V316          V317          V318          V319          V320 
 -6.710938649  -4.081881311   3.545716455 -26.600570652   1.406367563 
         V321          V322          V323          V324          V325 
  2.79

         V576          V577          V578          V579          V580 
  4.027340441  -0.552826709   2.729146093   4.769851534  -4.739664735 
         V581          V582          V583          V584          V585 
  1.925745515 -10.491719155   2.767991186  -0.360191806  -1.636569555 
         V586          V587          V588          V589          V590 
  1.056132989 -12.867653626   0.016555888   3.552664875   0.109862948 
         V591          V592          V593          V594          V595 
  5.034493177  -2.515997939   0.778915676  -3.897881168  24.431807786 
         V596          V597          V598          V599          V600 
  3.560455889   4.834448623   2.822389495   9.170885338   2.265236523 
         V601          V602          V603          V604          V605 
 -3.811333001   1.692964057  -2.195856703   0.969616247  -7.065672640 
         V606          V607          V608          V609          V610 
  9.177340732   1.701328629   4.435035514   2.501149186  -3.543822077 
      

  2.538253645   2.413230104  -7.733075488   2.531315064  -3.795917397 
         V866          V867          V868          V869          V870 
 -2.742004863  -2.774180701  -2.695833450 -15.239842822  -3.536071964 
         V871          V872          V873          V874          V875 
  1.841925038  -4.721773088   3.913334316  -1.560487554  -4.631427517 
         V876          V877          V878          V879          V880 
 -3.222693090   0.903307890   3.141105879   3.785482833  -1.379852132 
         V881          V882          V883          V884          V885 
 -1.589217497   2.916294415   7.025192304   8.327245740  -7.019922452 
         V886          V887          V888          V889          V890 
 -3.953363974  -5.637144368   2.490417662  -9.799539004   9.589641828 
         V891          V892          V893          V894          V895 
  4.219383412  -6.976695840   5.477135316  -3.892656378  -3.254306488 
         V896          V897          V898          V899          V900 
 -0.61

        V1151         V1152         V1153         V1154         V1155 
 -4.332751518   5.892386187   1.749692755  -3.735962360  -3.408705283 
        V1156         V1157         V1158         V1159         V1160 
  0.528185658  -7.796009529  -7.512452113   4.822895945   0.820704439 
        V1161         V1162         V1163         V1164         V1165 
  0.846710959  10.155014245   5.133908611  -7.273784879  -8.671039623 
        V1166         V1167         V1168         V1169         V1170 
 -3.948993661   6.407173152  -0.815315031   2.365915699  -5.185898172 
        V1171         V1172         V1173         V1174         V1175 
  5.551089645   7.172720830   0.381372651  -1.210085896   1.411052917 
        V1176         V1177         V1178         V1179         V1180 
  1.888956682  -1.033093903   3.144308695  -5.492652382   1.599398182 
        V1181         V1182         V1183         V1184         V1185 
 -2.200651226  -7.595577421  10.084062166  -3.087902945   3.050954858 
      

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



        V3173         V3174         V3175 
 -4.698694549   1.200709659 -11.173432121   4.764544641   2.614224243 
        V3176         V3177         V3178         V3179         V3180 
  0.111787293   6.780536823  -3.506028798 -14.548405938  -0.088372667 
        V3181         V3182         V3183         V3184         V3185 
  2.522797318   1.674175347  -5.110285604  -4.415402704   2.325335354 
        V3186         V3187         V3188         V3189         V3190 
 -2.766764637  -0.060522709   0.497200453  -5.906697610   2.477363042 
        V3191         V3192         V3193         V3194         V3195 
 -6.344123202  -2.241664301   2.890640701  -0.841098063  -3.131577815 
        V3196         V3197         V3198         V3199         V3200 
 -6.515544217   0.796257277  -7.387660977   3.625345063  -3.723331262 
        V3201         V3202         V3203         V3204         V3205 
  1.637822317   1.780354130  -5.798487322   2.526535802  -5.282026639 
        V3206         V3207       

 -6.534455997  -3.534831414   2.354042155  11.929792819  21.764154295 
        V3461         V3462         V3463         V3464         V3465 
 -4.093501358  20.141593918  -7.263934932  -2.524570631   4.015541289 
        V3466         V3467         V3468         V3469         V3470 
  2.815308335  -6.194084070   0.354290915   1.328806756  -3.902489036 
        V3471         V3472         V3473         V3474         V3475 
 11.484392048  -3.988918663  -5.063340648  15.580552516  -2.146336250 
        V3476         V3477         V3478         V3479         V3480 
  5.251669777   2.107328836   1.978267603  -5.770323454  13.239542091 
        V3481         V3482         V3483         V3484         V3485 
 -0.031483113  -7.929317244  -0.055249223   1.670964438  -4.223863256 
        V3486         V3487         V3488         V3489         V3490 
  2.493065516  -4.226302230  17.864750043   3.015935579   3.374900564 
        V3491         V3492         V3493         V3494         V3495 
  1.55

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



        V6016         V6017         V6018         V6019         V6020 
  3.843729995 -21.234786166  -3.369445404  16.917927046  -1.795454025 
        V6021         V6022         V6023         V6024         V6025 
  2.804617245  -4.239510953   0.931093468   7.335004305   3.333745986 
        V6026         V6027         V6028         V6029         V6030 
  3.941144345   2.112436668   2.507074834   1.874650279   9.513714356 
        V6031         V6032         V6033         V6034         V6035 
 -1.947614235  -0.955667532   5.414915428  -0.803355439   5.130872729 
        V6036         V6037         V6038         V6039         V6040 
  6.572137876  -5.718508560   3.565910319  -3.577069753   5.864266741 
        V6041         V6042         V6043         V6044         V6045 
 -4.064166692  -4.216821419  -4.568348399  -5.587117241  -5.078759959 
        V6046         V6047         V6048         V6049         V6050 
  1.154983649   3.913645794  -3.775801460   0.857976283  -1.978347266 
      

In [57]:
%%R
n_rxns <- length(rxns)
print(n_rxns)
result_idx <- 1
for(rxn_id_idx in seq(1:n_rxns)){
   rxn_id <- rxns[rxn_id_idx]
   rxn_pca <-
      prcomp(t(vst.count.mtx.train[rxn2ensembls.nls[[rxn_id]], ]), scale. = T)
   full_rxn_pca_results.nls[[rxn_id]] <- rxn_pca
   rxn_id_2_result_file_idx.nls[[rxn_id]] <- result_idx
   rxn_pca.nls[[rxn_id]] <-
      rxn_pca$x[, 1] # 1st principal component coordinate within this reaction-space for each sample
   if(mod(rxn_id_idx,100) == 0){
      print(paste("Processed ",rxn_id_idx,
                  " of ",n_rxns,
                  " reactions (",round((rxn_id_idx + 1)/n_rxns,digits = 3) * 100,"%)...",
                  sep=""))
      flush.console()
   }
   if(mod(rxn_id_idx,1000) == 0){
      print(paste("Storing PCA objects containing reactions ",rxn_id_idx-1000,
                  "-",rxn_id_idx,
                  " of ",n_rxns,
                  " reactions (",round((rxn_id_idx + 1)/n_rxns,digits = 3) * 100,"%)...",
                  sep=""))
      saveRDS(full_rxn_pca_results.nls,
              paste(OUT_DIR, "full_rxn_pca_results_nls",rxn_id_idx-1000,
                    "-",rxn_id_idx,".Rds", sep=""))
      full_rxn_pca_results.nls <- list()
      gc()
      result_idx <- result_idx + 1
   }
}

[1] 9844
[1] "Processed 100 of 9844 reactions (1%)..."
[1] "Processed 200 of 9844 reactions (2%)..."
[1] "Processed 300 of 9844 reactions (3.1%)..."
[1] "Processed 400 of 9844 reactions (4.1%)..."
[1] "Processed 500 of 9844 reactions (5.1%)..."
[1] "Processed 600 of 9844 reactions (6.1%)..."
[1] "Processed 700 of 9844 reactions (7.1%)..."
[1] "Processed 800 of 9844 reactions (8.1%)..."
[1] "Processed 900 of 9844 reactions (9.2%)..."
[1] "Processed 1000 of 9844 reactions (10.2%)..."
[1] "Storing PCA objects containing reactions 0-1000 of 9844 reactions (10.2%)..."
[1] "Processed 1100 of 9844 reactions (11.2%)..."
[1] "Processed 1200 of 9844 reactions (12.2%)..."
[1] "Processed 1300 of 9844 reactions (13.2%)..."
[1] "Processed 1400 of 9844 reactions (14.2%)..."
[1] "Processed 1500 of 9844 reactions (15.2%)..."
[1] "Processed 1600 of 9844 reactions (16.3%)..."
[1] "Processed 1700 of 9844 reactions (17.3%)..."
[1] "Processed 1800 of 9844 reactions (18.3%)..."
[1] "Processed 1900 of 9844 re

In [58]:
%%R
# store remaining PCA objects and removing from RAM
saveRDS(rxn_id_2_result_file_idx.nls,
        paste(OUT_DIR,"rxn_id_2_result_file_idx_nls.Rds",sep=""))
saveRDS(full_rxn_pca_results.nls,
        paste(OUT_DIR, "full_rxn_pca_results_nls.Rds", sep=""))
rm(full_rxn_pca_results.nls)
gc()

            used   (Mb) gc trigger   (Mb)  max used   (Mb)
Ncells   6991328  373.4   12953624  691.8  12953624  691.8
Vcells 257166467 1962.1  444386441 3390.4 361249692 2756.2


In [59]:
%%R
# compare informaction content of below files with pca plots or similar
saveRDS(rxn_pca.nls, paste(OUT_DIR, "rxn_pca_nls.Rds", sep = ""))
saveRDS(vst.count.mtx.train,
        paste(OUT_DIR, "vst_count_mtx_train.Rds", sep = ""))

In [60]:
%%R
for (cv_fold in names(cv_fold_indices)) {
  cur_cv_fold_indices <- cv_fold_indices[[cv_fold]]
  print(cur_cv_fold_indices)
  print("----------")
}

  [1]    9   10   42   58   64   72   76   90   92  104  105  113  123  137  163
 [16]  177  183  186  203  219  244  249  269  300  306  311  331  340  348  362
 [31]  366  368  370  371  374  380  389  418  435  447  454  473  488  516  519
 [46]  522  529  531  542  573  574  579  585  612  617  628  632  653  658  664
 [61]  674  675  677  687  708  711  712  714  765  791  794  799  816  823  824
 [76]  831  834  837  847  856  874  876  886  889  911  921  925  929  930  934
 [91]  944  960  969  993 1002 1021 1032 1034 1041 1047 1050 1053 1064 1070 1075
[106] 1090 1106 1129 1131 1133 1144 1152 1153 1171 1176 1188 1218 1226 1228 1231
[121] 1268 1271 1276 1290 1306 1320 1354 1362 1371 1397 1405 1417 1424 1425 1427
[136] 1430 1439 1473 1489 1497 1522 1532 1538 1540 1542 1549 1564 1569 1579 1580
[151] 1584 1594 1600 1607 1624 1638 1653 1656 1679 1683 1715 1729 1734 1765 1767
[166] 1769 1779 1783 1789 1792 1794 1801 1821 1822 1826 1833 1844 1846 1859 1866
[181] 1869 1870 1875 1890 18

[241] 2378 2381 2383 2387 2397 2414 2416 2426 2428 2439 2441 2444 2461 2474 2475
[256] 2476 2484 2490 2495 2522 2525 2526 2527 2552 2568 2582 2583 2606 2623 2657
[271] 2733 2736 2745 2749 2758 2764 2767 2783 2785 2809 2811 2812 2815 2827 2830
[286] 2845 2853 2855 2862 2865 2866 2869 2878 2888 2898 2927 2928 2932 2938 2942
[301] 2946 2948 2959 2963 2967 2990 2993 3017 3021 3023 3035 3046 3050 3051 3054
[316] 3069 3073 3075 3078 3083 3084 3097 3098 3107 3118 3124 3133 3140 3149 3163
[331] 3190 3200 3214 3221 3228 3230 3238 3244 3261 3262 3264 3269 3274 3278 3288
[346] 3316 3329 3353 3356 3369 3376 3386 3409 3426 3438 3465 3468 3471 3473 3475
[361] 3477 3479 3480 3493 3503 3514 3538 3545 3556 3568 3582 3584 3586 3592 3628
[376] 3643 3679 3685 3690 3697 3702 3708 3734 3740 3746 3757 3770 3777 3801 3823
[391] 3875 3886 3906 3920 3926 3927 3928 3934 3943 3945 3946 3949 3973 3979 3983
[406] 3986 3992 3999 4000 4001 4015 4033 4051 4052 4082 4109 4113 4131 4132 4135
[421] 4140 4149 4155 4160 41

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



  194  197  216  226  227  228  229  241  260  261  272
 [31]  279  293  304  307  308  312  314  318  324  328  342  364  381  419  427
 [46]  445  456  459  491  494  509  513  514  536  544  551  554  564  572  576
 [61]  588  592  604  608  659  660  665  681  693  717  719  728  737  747  748
 [76]  753  781  783  806  814  820  843  848  854  872  878  890  899  904  932
 [91]  946  957  967 1009 1012 1016 1023 1035 1037 1042 1076 1087 1096 1097 1110
[106] 1112 1130 1147 1162 1164 1168 1180 1185 1196 1204 1207 1220 1222 1261 1270
[121] 1281 1286 1287 1293 1297 1301 1315 1345 1348 1350 1370 1375 1380 1381 1385
[136] 1392 1406 1414 1433 1436 1451 1454 1474 1476 1477 1509 1518 1537 1541 1545
[151] 1563 1573 1577 1597 1631 1639 1641 1648 1668 1675 1676 1681 1693 1702 1704
[166] 1737 1747 1748 1757 1759 1772 1773 1787 1798 1805 1827 1835 1836 1855 1863
[181] 1876 1886 1889 1905 1909 1925 1927 1954 1960 1970 1978 1995 2001 2022 2038
[196] 2043 2044 2049 2055 2060 2062 2075 2078 2079 21

In [ ]:
%%R
# main loop
for (rxn_id_idx in seq(1:length(rxns))) {
   rxn_id <- rxns[rxn_id_idx]
   ensembl_ids <- rxn2ensembls.nls[[rxn_id]]
   
   mean_misclass_rate <- list()
   sum_ari <- 0
   
   for (cv_fold in names(cv_fold_indices)) {
      cur_cv_fold_indices <- cv_fold_indices[[cv_fold]]
      
      vst.count.mtx.train.cv_train <-
         vst.count.mtx.train[, -cur_cv_fold_indices] 
      vst.count.mtx.train.cv_test <-
         vst.count.mtx.train[, cur_cv_fold_indices] 
      
      GEO_model_training_tissue_detail.vec.train.cv_train <-
         GEO_model_training_tissue_detail.vec.train[-cur_cv_fold_indices] 
      GEO_model_training_tissue_detail.vec.train.cv_test <-
         GEO_model_training_tissue_detail.vec.train[cur_cv_fold_indices] 
       
      binary_GEO_model_training_tissue_detail_vec.test.cv_test_list <- list()
      for (tissue_annotation in binary_GEO_model_training_tissue_annotations) {
         binary_GEO_model_training_tissue_detail_vec.test.cv_test_list[[tissue_annotation]] <-
            (GEO_model_training_tissue_detail.vec.train.cv_test == tissue_annotation)
      }
      
      cv_train.expr_mat <-
         t(vst.count.mtx.train.cv_train[ensembl_ids, ])
      cv_test.expr_mat <-
         t(vst.count.mtx.train.cv_test[ensembl_ids, ])
      
      rxn_knn_calls <- class::knn(train = cv_train.expr_mat,
                                  test = cv_test.expr_mat,
                                  cl = GEO_model_training_tissue_detail.vec.train.cv_train)
      
      # calculate & store adjusted rand index
      cur_ari <- pdfCluster::adj.rand.index(rxn_knn_calls,
                                            GEO_model_training_tissue_detail.vec.train.cv_test)
      sum_ari <- cur_ari + sum_ari
      
      # for each tissue, calculate misclassification rate
      for (tissue_annotation in binary_GEO_model_training_tissue_annotations) {
         cur_rxn_knn_calls <- (rxn_knn_calls == tissue_annotation)
            
         tab <- table(cur_rxn_knn_calls,
                      binary_GEO_model_training_tissue_detail_vec.test.cv_test_list[[tissue_annotation]])

         cur_misclass_rate <- 1 - sum(diag(tab)) / sum(tab)
         sum_misclass_rate <- cur_misclass_rate
         if (!is.null(mean_misclass_rate[[tissue_annotation]])) {
           sum_misclass_rate <- sum_misclass_rate + mean_misclass_rate[[tissue_annotation]]
         }
         mean_misclass_rate[[tissue_annotation]] <- sum_misclass_rate
      }
   }
   for(tissue_annotation in binary_GEO_model_training_tissue_annotations){
      mean_misclass_rate[[tissue_annotation]] <- (mean_misclass_rate[[tissue_annotation]] / N_FOLDS)
   }
   mean_ari <- sum_ari / N_FOLDS
   ecount <- length(ensembl_ids)
   
   rxn_knn_misclass_rate.nls[[rxn_id]] <- mean_misclass_rate
   assign("mean_misclass_rate",NULL,envir = .GlobalEnv)
   
   rxn_knn_ari.nls[[rxn_id]] <- mean_ari
   rxn_knn_ecount.nls[[rxn_id]] <- ecount
   
   count <- count + 1
   if (mod(count, 10) == 0) {
      print(
         paste(
            "Last RXN_ID = ",
            rxn_id,
            ": Last ARI = ",
            mean_ari,
            ": Last ECOUNT = ",
            ecount,
            ": Last Lung MISCLASS = ",
            rxn_knn_misclass_rate.nls[[rxn_id]][["Lung"]],
            ": Last Uterus MISCLASS = ",
            rxn_knn_misclass_rate.nls[[rxn_id]][["Uterus"]],
            ". Now ",
            round(1.0 - count / length(rxns),3) * 100,
            "% remaining..."
         )
      )
      flush.console()
   }
}

[1] "Last RXN_ID =  R-MMU-749456 : Last ARI =  0.968870364551919 : Last ECOUNT =  202 : Last Lung MISCLASS =  0.00222576400737889 : Last Uterus MISCLASS =  0.000158982511923689 . Now  99.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-9021601 : Last ARI =  0.948114607967421 : Last ECOUNT =  24 : Last Lung MISCLASS =  0.00222576080033042 : Last Uterus MISCLASS =  0 . Now  99.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-3108203 : Last ARI =  0.898691556421089 : Last ECOUNT =  15 : Last Lung MISCLASS =  0.00477050891539268 : Last Uterus MISCLASS =  0.000477962589659187 . Now  99.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-9838627 : Last ARI =  0.504587878132722 : Last ECOUNT =  4 : Last Lung MISCLASS =  0.0162251443923782 : Last Uterus MISCLASS =  0.0034971218151387 . Now  99.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-193679 : Last ARI =  0.243069380002332 : Last ECOUNT =  2 : Last Lung MISCLASS =  0.0551821127375379 : Last Uterus MISCLASS =  0.00874433236563511 . Now  99.5 % remaining..."
[1

[1] "Last RXN_ID =  R-MMU-8849353 : Last ARI =  0.907832227727173 : Last ECOUNT =  42 : Last Lung MISCLASS =  0.00333762815968326 : Last Uterus MISCLASS =  0.000159489633173848 . Now  95.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-5624492 : Last ARI =  0.739810785380511 : Last ECOUNT =  6 : Last Lung MISCLASS =  0.0122503147556614 : Last Uterus MISCLASS =  0.00254676857097906 . Now  95.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-5672978 : Last ARI =  0.928386037013143 : Last ECOUNT =  34 : Last Lung MISCLASS =  0.00238575756246964 : Last Uterus MISCLASS =  0.000318218180713503 . Now  95.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-9657599 : Last ARI =  0.928386037013143 : Last ECOUNT =  34 : Last Lung MISCLASS =  0.00238575756246964 : Last Uterus MISCLASS =  0.000318218180713503 . Now  95.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-186765 : Last ARI =  0.777696347984483 : Last ECOUNT =  5 : Last Lung MISCLASS =  0.00620363076891196 : Last Uterus MISCLASS =  0.0036566082412397 . Now  95.

[1] "Last RXN_ID =  R-MMU-5621571 : Last ARI =  0.21988286623017 : Last ECOUNT =  2 : Last Lung MISCLASS =  0.0182881470502696 : Last Uterus MISCLASS =  0.0143135488517636 . Now  91.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-8867756 : Last ARI =  0.951242896842057 : Last ECOUNT =  87 : Last Lung MISCLASS =  0.00238676616001063 : Last Uterus MISCLASS =  0.000317461117311957 . Now  91.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-2213243 : Last ARI =  0.868914129794015 : Last ECOUNT =  14 : Last Lung MISCLASS =  0.0066767865659367 : Last Uterus MISCLASS =  0.00158983316372676 . Now  91.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-163096 : Last ARI =  0.884460445824586 : Last ECOUNT =  18 : Last Lung MISCLASS =  0.00333915033866685 : Last Uterus MISCLASS =  0.000635930047694755 . Now  91.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-5624952 : Last ARI =  0.937657587148553 : Last ECOUNT =  53 : Last Lung MISCLASS =  0.00206728218727589 : Last Uterus MISCLASS =  0.000159489633173848 . Now  91.4

[1] "Last RXN_ID =  R-MMU-5610725 : Last ARI =  0.847035767539438 : Last ECOUNT =  7 : Last Lung MISCLASS =  0.00810915371328587 : Last Uterus MISCLASS =  0.00142984123520801 . Now  87.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-5690996 : Last ARI =  0.861590112431804 : Last ECOUNT =  25 : Last Lung MISCLASS =  0.00413381698221209 : Last Uterus MISCLASS =  0.000795672837734729 . Now  87.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-6782131 : Last ARI =  0.882049764270711 : Last ECOUNT =  43 : Last Lung MISCLASS =  0.00508771204603899 : Last Uterus MISCLASS =  0.000477707813887351 . Now  87.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-75850 : Last ARI =  0.885318258117986 : Last ECOUNT =  46 : Last Lung MISCLASS =  0.00477076128012237 : Last Uterus MISCLASS =  0.000477707813887351 . Now  87.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-76576 : Last ARI =  0.876065598017248 : Last ECOUNT =  23 : Last Lung MISCLASS =  0.00540771040393789 : Last Uterus MISCLASS =  0.000477457871735953 . Now  87

In [ ]:
%%R
saveRDS(
   rxn_knn_misclass_rate.nls,
   paste(OUT_DIR, "toi_rxn_knn_misclass_rate_nls.Rds", sep = "")
)
saveRDS(rxn_knn_ari.nls,
        paste(OUT_DIR, "toi_rxn_knn_ari_nls.Rds", sep = ""))
saveRDS(rxn_knn_ecount.nls,
        paste(OUT_DIR, "toi_rxn_knn_ecount_nls.Rds", sep = ""))

In [ ]:
%%R
d <- data.frame(
   RXN_ID = names(rxn2ensembls.nls),
   MISCLASS = unlist(rxn_knn_misclass_rate.nls),
   ARI = unlist(rxn_knn_ari.nls),
   ECOUNT = unlist(rxn_knn_ecount.nls)
)
print(d)
saveRDS(d, paste(OUT_DIR, "toi_summary_df.Rds", sep = ""))

In [ ]:
%%R
library(htmlwidgets)
min_misclass_pca <-
   prcomp(t(vst.count.mtx.train[rxn2ensembls.nls[["R-MMU-450466"]], ]), scale. = T)
pca.d <- data.frame(
   PC1 = min_misclass_pca$x[, 1],
   PC2 = min_misclass_pca$x[, 2],
   PC3 = min_misclass_pca$x[, 3],
   Section = GEO_model_training_tissue_detail.vec.train
)
ggplot(pca.d) +
   geom_point(aes(x = PC1, y = PC2, colour = Section)) +
   theme_bw()gai
p <- plot_ly(
   x = pca.d$PC1,
   y = pca.d$PC2,
   z = pca.d$PC3,
   type = "scatter3d",
   mode = "markers",
   color = pca.d$Section,
   size = 1
)
saveWidget(p, "plotly_chart_80.html", selfcontained = TRUE)